In [2]:
import pandas as pd
import numpy as np
import os

# File name
### 저장하고자 하는 excel 파일 이름과 해당되는 조건 입력

In [3]:
file_path = 'LERRDFWFV_167set+Testdataset_4set_V002.xlsx'

# file_path에 따라 변수 결정 (N형만 생각)

if file_path == '2020_LER_20201008_V008.xlsx':
    num_input = 3
    onehot = 0
    num_output = 6
    num_in_cycle = 50
    num_of_cycle = 127
    num_total = num_in_cycle * num_of_cycle
    x_cols = "D:F"
    y_cols = "H:P"
    header = 0

elif file_path == '2020_LER_20201102_testset_V04.xlsx':
    num_input = 3
    onehot = 0
    num_output = 6
    num_in_cycle = 250
    num_of_cycle = 10
    num_total = num_in_cycle*num_of_cycle
    x_cols = "D:F"
    y_cols = "H:P"
    header = 0
    
elif file_path == '2020_RDFWFV_20201222_V10.xlsx':
    num_input = 4 # RDF, WFV, RDF+WFV
    onehot = 2
    num_output = 6
    num_in_cycle = 50
    num_of_cycle = [50, 50, 100]
    num_total = [x*num_in_cycle for x in num_of_cycle]
    header = 0
      
elif file_path == '2021_RDFWFV_20210107.xlsx':
    num_input = 4
    one_hot = 2
    num_output = 6
    num_in_cycle = 250
    num_of_cycle = [5, 5, 6]
    num_total = sum(num_of_cycle)*num_in_cycle
    x_cols = "C:F"
    y_cols = "G:N"
    onehot = "A:B"
    header = 0

    
elif file_path == 'LERRDFWFV_167set+Testdataset_4set_V002.xlsx':
    num_input = 6
    num_output = 6
    
    num_in_cycle = 50
    num_of_cycle = 167
    num_total = num_in_cycle * num_of_cycle
    x_cols = "B:G"
    y_cols = "H:P"
    header = 0
    
    test_num_in_cycle = 250
    test_num_of_cycle = 4
    test_num_total = test_num_of_cycle * test_num_in_cycle
    test_x_cols = "B:I"
    test_y_cols = "J:R"
    test_x_header = 2 
    test_y_header = 1

In [4]:
def mean_cov(y_all, num_in_cycle, num_of_cycle, num_output):
    
    print("Y_all size: ", y_all.shape)
    
    y_mean_cov_num = int(2*num_output+((num_output)**2-(num_output))/2)
    # print(y_mean_cov_num)

    y_mean_cov = np.zeros((num_of_cycle, y_mean_cov_num))
    # print(y_mean_cov.shape)
    
    for i in range(num_of_cycle):
        #print(i)        
#        print(i*num_in_cycle, (i+1)*num_in_cycle)
        temp = y_all[i*num_in_cycle:(i+1)*num_in_cycle,:]
    
        mean_y = np.mean(temp, axis=0)
        cov_y = np.cov(temp.T)
        # print(cov_y)
        
        # mean
        y_mean_cov[i, :num_output] = mean_y
        # print(y_mean_cov)
        # diagonal
        y_mean_cov[i, num_output:num_output*2] = np.diagonal(cov_y)
        # print(y_mean_cov)
        
        # covariance
        cnt = num_output*2
        for j in range(1, num_output):
            y_mean_cov[i,cnt:cnt+j] = cov_y[j,:j]
            cnt += j
            
    return y_mean_cov

In [88]:
file_path = 'LERRDFWFV_167set+Testdataset_4set_V002.xlsx'

# Train
# Read data
data_x = pd.read_excel('../'+file_path, sheet_name='LERRDFWFV data', usecols=x_cols, nrows=num_total+1, header=header)
data_y = pd.read_excel('../'+file_path, sheet_name='LERRDFWFV data', usecols=y_cols, nrows=num_total+1, header=header)


# DATA_X DATA_Y preprocessing
# Y: IDLO, IDHI, Idlinlo
data_y = data_y.drop('Idhi', axis=1)
data_y = data_y.drop('Idlo', axis=1)
data_y = data_y.drop('Idlinlo', axis=1)

# rearrange Y cols ( same with RDFWFV_20201222V_10 )
cols = list(data_y.columns.values)
print(cols)
# this file's Y col order = ['Vtlin', 'Idlin', 'Ioff', 'Vtsat', 'Idsat', 'SS']
cols = ['Ioff' ,'Vtlin', 'Vtsat', 'Idlin', 'Idsat', 'SS']
data_y = data_y[cols]
print(list(data_y.columns.values))
# check
print(data_y)
print(data_x)

# PANDAS to NUMPY

num_of_cycle = int(len(data_y)/num_in_cycle)
num_total = len(data_y)
print('num_of_cycle', num_of_cycle, 'num_in_cycle', num_in_cycle, 'num_total', num_total)

X_all , Y_all = np.zeros((num_total, num_input)), np.zeros((num_total, num_output))
X_per_cycle, Y_per_cycle = np.zeros((num_of_cycle, num_input)), np.zeros((num_of_cycle, num_output))

# X_per_cycle
for i in range(num_of_cycle):
    X_per_cycle[i] = data_x[i*num_in_cycle:i*num_in_cycle+1].values

# X_all
X_all = np.repeat(X_per_cycle,num_in_cycle,axis=0)

# Y_all
for i in range(num_of_cycle):
    Y_all[i*num_in_cycle:(i+1)*num_in_cycle] = data_y[i*num_in_cycle:(i+1)*num_in_cycle].values

# Y_per_cycle    
for i in range(num_of_cycle):
    Y_per_cycle[i] = np.mean(Y_all[i*num_in_cycle:(i+1)*num_in_cycle],axis=0)

Y_per_mean_cov = mean_cov(Y_all, num_in_cycle, num_of_cycle, num_output)
print(Y_per_mean_cov.shape)
# for i in range(len(Y_per_mean_cov)):
#     print(Y_per_mean_cov[i])
    
print("============ Data load =============")

print("X data shape: ", X_all.shape, "X per cycle data shape:", X_per_cycle.shape)
print("Y data shape: ", Y_all.shape, "Y per cycle data shape:", Y_per_cycle.shape)  
print("any nan in X?: ", np.argwhere(np.isnan(X_all)))
print("any nan in Y?: ", np.argwhere(np.isnan(Y_all)))  

print("============ Data save =============")
    
data = []
data.append(X_all)
data.append(Y_all)
data.append(X_per_cycle)
data.append(Y_per_cycle)
data.append(Y_per_mean_cov)
    
data = np.array(data)

print("data : data_all")
print("[saved] X data : ({},{}) X per cycle data shape : ({},{})".format(len(data[0]), len(data[0][0]), len(data[2]), len(data[2][0])))                                                                  
print("[saved] Y data : ({},{}) Y per cycle data shape : ({},{}) Y mean cov data shape : ({},{})".format(len(data[1]), len(data[1][0]), len(data[3]), len(data[3][0]), len(data[4]), len(data[4][0])))  
np.save('./train_'+file_path, data)

['Vtlin', 'Idlin', 'Ioff', 'Vtsat', 'Idsat', 'SS']
['Ioff', 'Vtlin', 'Vtsat', 'Idlin', 'Idsat', 'SS']
              Ioff  Vtlin  Vtsat     Idlin     Idsat      SS
0     1.827000e-12  0.364  0.354  0.000010  0.000020  62.548
1     6.364000e-11  0.276  0.255  0.000012  0.000030  63.291
2     6.022000e-12  0.339  0.329  0.000010  0.000022  63.951
3     6.349000e-12  0.352  0.321  0.000010  0.000021  63.713
4     1.945000e-12  0.362  0.354  0.000010  0.000020  62.527
...            ...    ...    ...       ...       ...     ...
8345  8.583000e-10  0.319  0.233  0.000009  0.000038  79.864
8346  1.632000e-09  0.320  0.222  0.000010  0.000038  84.598
8347  4.421000e-10  0.360  0.264  0.000010  0.000034  81.315
8348  3.484000e-10  0.336  0.262  0.000010  0.000035  78.589
8349  3.388000e-09  0.301  0.195  0.000011  0.000044  86.799

[8350 rows x 6 columns]
           amp       corx       cory                    NSD  \
0     0.006057  37.343882  35.771567  186532361476567990272   
1          NaN 

In [89]:
file_path = 'LERRDFWFV_167set+Testdataset_4set_V002.xlsx'

# Test
# Read data
data_x = pd.read_excel('../'+file_path, sheet_name='Testset', usecols=test_x_cols, nrows=test_num_total+1, header=test_x_header)
data_y = pd.read_excel('../'+file_path, sheet_name='Testset', usecols=test_y_cols, nrows=test_num_total+1, header=test_y_header)

# DATA_X DATA_Y preprocessing
# Y: IDLO, IDHI, Idlinlo
# X: SDSF, CHSF
data_y = data_y.drop('Idhi', axis=1)
data_y = data_y.drop('Idlo', axis=1)
data_y = data_y.drop('Idlinlo', axis=1)
data_y = data_y.drop([0])
data_x = data_x.drop('SDSF', axis=1)
data_x = data_x.drop('CHSF', axis=1)

# rearrange Y cols ( same with RDFWFV_20201222V_10 )
cols = list(data_y.columns.values)
print(cols)
# this file's Y col order = ['Vtlin', 'Idlin', 'Ioff', 'Vtsat', 'Idsat', 'SS']
cols = ['Ioff' ,'Vtlin', 'Vtsat', 'Idlin', 'Idsat', 'SS']
data_y = data_y[cols]
print(list(data_y.columns.values))
# check
print(data_y)
print(data_x)

# PANDAS to NUMPY

print('num_of_cycle', test_num_of_cycle, 'num_in_cycle', test_num_in_cycle, 'num_total', test_num_total)

X_all , Y_all = np.zeros((test_num_total, num_input)), np.zeros((test_num_total, num_output))
X_per_cycle, Y_per_cycle = np.zeros((test_num_of_cycle, num_input)), np.zeros((test_num_of_cycle, num_output))

# X_per_cycle
for i in range(test_num_of_cycle):
    X_per_cycle[i] = data_x[i*test_num_in_cycle:i*test_num_in_cycle+1].values

# X_all
X_all = np.repeat(X_per_cycle,test_num_in_cycle,axis=0)

# Y_all
for i in range(test_num_of_cycle):
    Y_all[i*test_num_in_cycle:(i+1)*test_num_in_cycle] = data_y[i*test_num_in_cycle:(i+1)*test_num_in_cycle].values

# Y_per_cycle    
for i in range(test_num_of_cycle):
    Y_per_cycle[i] = np.mean(Y_all[i*test_num_in_cycle:(i+1)*test_num_in_cycle],axis=0)

Y_per_mean_cov = mean_cov(Y_all, test_num_in_cycle, test_num_of_cycle, num_output)
print(Y_per_mean_cov.shape)
# for i in range(len(Y_per_mean_cov)):
#     print(Y_per_mean_cov[i])
    
print("============ Data load =============")

print("X data shape: ", X_all.shape, "X per cycle data shape:", X_per_cycle.shape)
print("Y data shape: ", Y_all.shape, "Y per cycle data shape:", Y_per_cycle.shape)  
print("any nan in X?: ", np.argwhere(np.isnan(X_all)))
print("any nan in Y?: ", np.argwhere(np.isnan(Y_all)))  

print("============ Data save =============")
    
data = []
data.append(X_all)
data.append(Y_all)
data.append(X_per_cycle)
data.append(Y_per_cycle)
data.append(Y_per_mean_cov)
    
data = np.array(data)

print("data : data_all")
print("[saved] X data : ({},{}) X per cycle data shape : ({},{})".format(len(data[0]), len(data[0][0]), len(data[2]), len(data[2][0])))                                                                  
print("[saved] Y data : ({},{}) Y per cycle data shape : ({},{}) Y mean cov data shape : ({},{})".format(len(data[1]), len(data[1][0]), len(data[3]), len(data[3][0]), len(data[4]), len(data[4][0])))  
np.save('./test_'+file_path, data)

['Vtlin', 'Idlin', 'Ioff', 'Vtsat', 'Idsat', 'SS']
['Ioff', 'Vtlin', 'Vtsat', 'Idlin', 'Idsat', 'SS']
              Ioff  Vtlin  Vtsat     Idlin     Idsat       SS
1     1.335000e-10  0.273  0.242  0.000010  0.000042   66.438
2     7.647000e-08  0.135  0.071  0.000009  0.000039   93.191
3     4.510000e-08  0.175  0.097  0.000009  0.000037   95.249
4     3.695000e-08  0.173  0.101  0.000010  0.000038   90.908
5     7.220000e-08  0.159  0.072  0.000009  0.000038   91.135
...            ...    ...    ...       ...       ...      ...
996   1.360000e-08  0.191  0.144  0.000006  0.000027   89.054
997   7.230000e-09  0.207  0.165  0.000007  0.000027   85.859
998   7.750000e-08  0.126  0.078  0.000007  0.000031  101.642
999   1.210000e-08  0.206  0.144  0.000006  0.000027   86.326
1000  4.310000e-08   0.15  0.101  0.000007  0.000030   96.717

[1000 rows x 6 columns]
           amp       corx       cory           NSD           NCH       AGS
0     0.553809  68.084527  47.617352  1.735235e+20  9.

In [90]:
# 검증의 시간

data = np.load('train_'+file_path+'.npy', allow_pickle=True)
    
X_all, Y_all, X_per_cycle, Y_per_cycle = data[0], data[1], data[2], data[3]

print(data[0].shape, data[1].shape, data[2].shape, data[3].shape)

for i in range(X_per_cycle.shape[0]):
    print(X_per_cycle[i])
    
data = np.load('test_'+file_path+'.npy', allow_pickle=True)
    
X_all, Y_all, X_per_cycle, Y_per_cycle = data[0], data[1], data[2], data[3]

print(data[0].shape, data[1].shape, data[2].shape, data[3].shape)

for i in range(X_per_cycle.shape[0]):
    print(X_per_cycle[i])

(8350, 6) (8350, 6) (167, 6) (167, 6)
[6.05723977e-03 3.73438823e+01 3.57715673e+01 1.86532361e+20
 1.71413204e+17 4.11566198e-03]
[6.25206740e-01 3.75225389e+01 8.94337783e+01 8.07879750e+19
 1.03273454e+16 7.30117759e-03]
[2.40043915e-01 9.08263181e+01 5.53214772e+01 2.55547718e+19
 6.52449856e+16 4.51820726e-03]
[2.01237089e-01 3.25764473e+01 2.50807251e+01 1.43476009e+20
 9.91774229e+17 2.87429700e-03]
[4.36863952e-01 8.44306899e+01 8.59133682e+01 3.46754998e+19
 1.71460942e+17 6.32027792e-03]
[6.43577242e-02 6.43478103e+01 9.03058438e+01 5.78770910e+19
 4.64354206e+16 4.48863618e-03]
[5.99769687e-01 9.42606222e+01 5.57622071e+01 1.65281530e+19
 4.11703823e+15 4.54056731e-03]
[2.22289512e-01 1.76836727e+01 2.23198590e+01 1.30476992e+20
 7.86152857e+16 2.47941800e-03]
[7.68049119e-01 7.05418243e+01 9.10766048e+01 1.09407952e+20
 5.83575953e+17 8.07013310e-03]
[2.50745514e-01 9.16543683e+01 7.86494139e+01 4.05685823e+19
 2.08144115e+17 5.20527339e-03]
[6.78753129e-02 9.42488443e+01 6

# Log & remove outliers

In [11]:
# 검증의 시간

file_path = 'LERRDFWFV_167set+Testdataset_4set_V002.xlsx'

data = np.load('train_'+file_path+'.npy', allow_pickle=True)
X_all, Y_all, X_per_cycle, Y_per_cycle, Y_per_mean_cov = data[0], data[1], data[2], data[3], data[4]

print(data[0].shape, data[1].shape, data[2].shape, data[3].shape)

# 0. coefficient
num_in_cycle = 50
num_of_cycle = 167
num_output = 6
log_col = [3,4] # col 4,5 log

# 1. ood
ood = np.where(X_per_cycle[:,4] >0.25e19)
print('ood index', ood[0])
print('ood value', X_per_cycle[ood][:,4])

ood_all = []
for j in ood[0]:
    for i in range(j*num_in_cycle, (j+1)*num_in_cycle):
        ood_all.append(i)
# print('ood_all', ood_all)

u_X_all = np.delete(X_all, ood_all, axis=0)
u_Y_all = np.delete(Y_all, ood_all, axis=0)
u_X_per_cycle = np.delete(X_per_cycle, ood[0], axis=0)
u_Y_per_cycle = np.delete(Y_per_cycle, ood[0], axis=0)

# 2. log
for col in log_col:
    u_X_all[:,col] = np.log(u_X_all[:,col])
    u_X_per_cycle[:,col] = np.log(u_X_per_cycle[:,col])

# u_Y_per_mean_cov = np.delete(Y_per_mean_Cov, ood[0], axis=0)

u_Y_per_mean_cov = mean_cov(u_Y_all, num_in_cycle, num_of_cycle, num_output)
print(u_Y_per_mean_cov.shape)


print("X data shape: ", u_X_all.shape, "X per cycle data shape:", u_X_per_cycle.shape)
print("Y data shape: ", u_Y_all.shape, "Y per cycle data shape:", u_Y_per_cycle.shape)  
print("any nan in X?: ", np.argwhere(np.isnan(u_X_all)))
print("any nan in Y?: ", np.argwhere(np.isnan(u_Y_all)))  

print("============ Data save =============")
    
data = []
data.append(u_X_all)
data.append(u_Y_all)
data.append(u_X_per_cycle)
data.append(u_Y_per_cycle)
data.append(u_Y_per_mean_cov)
    
data = np.array(data)

print("data : data_all")
print("[saved] X data : ({},{}) X per cycle data shape : ({},{})".format(len(data[0]), len(data[0][0]), len(data[2]), len(data[2][0])))                                                                  
print("[saved] Y data : ({},{}) Y per cycle data shape : ({},{}) Y mean cov data shape : ({},{})".format(len(data[1]), len(data[1][0]), len(data[3]), len(data[3][0]), len(data[4]), len(data[4][0])))  
np.save('./updated_train_'+file_path, data)


## Test data

data = np.load('test_'+file_path+'.npy', allow_pickle=True)
    
X_all, Y_all, X_per_cycle, Y_per_cycle, Y_per_mean_cov = data[0], data[1], data[2], data[3], data[4]

print(data[0].shape, data[1].shape, data[2].shape, data[3].shape)

for col in log_col:
    X_all[:,col] = np.log(X_all[:,col])
    X_per_cycle[:,col] = np.log(X_per_cycle[:,col])

print("============ Data save =============")
    
data = []
data.append(X_all)
data.append(Y_all)
data.append(X_per_cycle)
data.append(Y_per_cycle)
data.append(Y_per_mean_cov)
    
data = np.array(data)

print("data : data_all")
print("[saved] X data : ({},{}) X per cycle data shape : ({},{})".format(len(data[0]), len(data[0][0]), len(data[2]), len(data[2][0])))                                                                  
print("[saved] Y data : ({},{}) Y per cycle data shape : ({},{}) Y mean cov data shape : ({},{})".format(len(data[1]), len(data[1][0]), len(data[3]), len(data[3][0]), len(data[4]), len(data[4][0])))  
np.save('./updated_test_'+file_path, data)


(8350, 6) (8350, 6) (167, 6) (167, 6)
ood index [ 71  81  83  85  92  93 117 159]
ood value [2.71044428e+18 1.19047274e+19 1.54034399e+19 3.18923798e+18
 8.15306189e+18 1.45972742e+19 6.79569671e+18 6.69939758e+18]
Y_all size:  (7950, 6)
(167, 27)
X data shape:  (7950, 6) X per cycle data shape: (159, 6)
Y data shape:  (7950, 6) Y per cycle data shape: (159, 6)
any nan in X?:  []
any nan in Y?:  []
============ Data save =============
data : data_all
[saved] X data : (7950,6) X per cycle data shape : (159,6)
[saved] Y data : (7950,6) Y per cycle data shape : (159,6) Y mean cov data shape : (167,27)
(1000, 6) (1000, 6) (4, 6) (4, 6)
============ Data save =============
data : data_all
[saved] X data : (1000,6) X per cycle data shape : (4,6)
[saved] Y data : (1000,6) Y per cycle data shape : (4,6) Y mean cov data shape : (4,27)


/home/jihwankwak/ENTER/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: Degrees of freedom <= 0 for slice


In [10]:
# 검증의 시간

data = np.load('updated_train_'+file_path+'.npy', allow_pickle=True)
    
X_all, Y_all, X_per_cycle, Y_per_cycle = data[0], data[1], data[2], data[3]

print(data[0].shape, data[1].shape, data[2].shape, data[3].shape)

for i in range(X_per_cycle.shape[0]):
    print(X_per_cycle[i])
    
data = np.load('updated_test_'+file_path+'.npy', allow_pickle=True)
    
X_all, Y_all, X_per_cycle, Y_per_cycle = data[0], data[1], data[2], data[3]

print(data[0].shape, data[1].shape, data[2].shape, data[3].shape)

for i in range(X_per_cycle.shape[0]):
    print(X_per_cycle[i])

(7950, 6) (7950, 6) (159, 6) (159, 6)
[6.05723977e-03 3.73438823e+01 3.57715673e+01 4.66751364e+01
 3.96828534e+01 4.11566198e-03]
[6.25206740e-01 3.75225389e+01 8.94337783e+01 4.58383598e+01
 3.68735717e+01 7.30117759e-03]
[2.40043915e-01 9.08263181e+01 5.53214772e+01 4.46873557e+01
 3.87169256e+01 4.51820726e-03]
[2.01237089e-01 3.25764473e+01 2.50807251e+01 4.64126995e+01
 4.14382719e+01 2.87429700e-03]
[4.36863952e-01 8.44306899e+01 8.59133682e+01 4.49925651e+01
 3.96831319e+01 6.32027792e-03]
[6.43577242e-02 6.43478103e+01 9.03058438e+01 4.55048533e+01
 3.83768389e+01 4.48863618e-03]
[5.99769687e-01 9.42606222e+01 5.57622071e+01 4.42515968e+01
 3.59539104e+01 4.54056731e-03]
[2.22289512e-01 1.76836727e+01 2.23198590e+01 4.63177286e+01
 3.89033426e+01 2.47941800e-03]
[7.68049119e-01 7.05418243e+01 9.10766048e+01 4.61416153e+01
 4.09079510e+01 8.07013310e-03]
[2.50745514e-01 9.16543683e+01 7.86494139e+01 4.51495256e+01
 3.98770071e+01 5.20527339e-03]
[6.78753129e-02 9.42488443e+01 6

# Data save
### 동일한 폴더에 .npy가 저장됨

# 2020_LER_20201008_V008

In [8]:
# file_path version : 2020_LER_20201008_V006.xlsx

data_x = pd.read_excel('../'+file_path, sheet_name='Generated DATAs', usecols=x_cols, nrows=num_total+1, header=header)
data_y = pd.read_excel('../'+file_path, sheet_name='Generated DATAs', usecols=y_cols, nrows=num_total+1, header=header)

# one-hot encoding (나중에 1, 0 을 인위적으로 넣어주기)
X_all , Y_all = np.zeros((num_total, num_input)), np.zeros((num_total, num_output))
X_per_cycle, Y_per_cycle = np.zeros((num_of_cycle, num_input)), np.zeros((num_of_cycle, num_output))

# DATA_X DATA_Y preprocessing

# Y: IDLO, IDHI, DIBL
data_y = data_y.drop('IDLO', axis=1)
data_y = data_y.drop('IDHI', axis=1)
data_y = data_y.drop('DIBL(mV)', axis=1)
print(data_x)
# print(data_y)

# PANDAS TO NUMPY
# X_per_cycle
for i in range(num_of_cycle):
    X_per_cycle[i] = data_x[i*num_in_cycle:i*num_in_cycle+1].values

# X_all
X_all = np.repeat(X_per_cycle,num_in_cycle,axis=0)


# Y_all
for i in range(num_of_cycle):
    Y_all[i*num_in_cycle:(i+1)*num_in_cycle] = data_y[i*num_in_cycle:(i+1)*num_in_cycle].values

# Y_per_cycle    
for i in range(num_of_cycle):
    Y_per_cycle[i] = np.mean(Y_all[i*num_in_cycle:(i+1)*num_in_cycle],axis=0)

Y_per_mean_cov = mean_cov(Y_all, num_in_cycle, num_of_cycle, num_output)
print(Y_per_mean_cov.shape)
for i in range(len(Y_per_mean_cov)):
    print(Y_per_mean_cov[i])
    
print("============ Data load =============")

print("X data shape: ", X_all.shape, "X per cycle data shape:", X_per_cycle.shape)
print("Y data shape: ", Y_all.shape, "Y per cycle data shape:", Y_per_cycle.shape)  
print("any nan in X?: ", np.argwhere(np.isnan(X_all)))
print("any nan in Y?: ", np.argwhere(np.isnan(Y_all)))

print("============ Data save =============")
data = []
data.append(X_all)
data.append(Y_all)
data.append(X_per_cycle)
data.append(Y_per_cycle)
data.append(Y_per_mean_cov)

print()
    
data = np.array(data)
print("saved data shape: ", data.shape)
print(file_path)
np.save('./'+file_path, data)

          amp.     corr.x      corr.y
0     0.193163  61.002273  110.617897
1          NaN        NaN         NaN
2          NaN        NaN         NaN
3          NaN        NaN         NaN
4          NaN        NaN         NaN
...        ...        ...         ...
6345       NaN        NaN         NaN
6346       NaN        NaN         NaN
6347       NaN        NaN         NaN
6348       NaN        NaN         NaN
6349       NaN        NaN         NaN

[6350 rows x 3 columns]
Y_all size:  (6350, 6)
(127, 27)
[ 2.25020000e-12  4.29014000e-05  1.66996000e-05  3.40960000e-01
  3.48600000e-01  6.16506000e+01  8.57633878e-26  3.24849020e-13
  1.11934531e-13  6.85551020e-06  4.85714286e-06  1.72450204e-02
  1.02302163e-19  8.12580408e-20  1.41867918e-13 -7.57297959e-16
 -9.34432653e-10 -7.22873469e-10 -6.33612245e-16 -7.68000000e-10
 -6.09755102e-10  5.67755102e-06  3.73673673e-14  4.47430204e-08
  3.70135102e-08 -3.24077551e-04 -2.73612245e-04]
[ 2.26560000e-12  3.95636000e-05  1.58274000e-

# Data load
### 저장한 .npy를 불러서 잘 저장했는지 확인

In [11]:
# 검증의 시간

data = np.load(file_path+'.npy', allow_pickle=True)
    
X_all, Y_all, X_per_cycle, Y_per_cycle = data[0], data[1], data[2], data[3]

print(data[0].shape, data[1].shape, data[2].shape, data[3].shape)

for i in range(X_per_cycle.shape[0]):
    print(X_per_cycle[i])

(6350, 3) (6350, 6) (127, 3) (127, 6)
[  0.19316255  61.00227254 110.61789695]
[  0.71479321  30.46485839 164.03098759]
[  0.36639042  45.71598335 128.60254701]
[ 0.19156989 92.07622363 49.23479694]
[  0.64566003  28.77580661 181.68354021]
[  0.24976869  49.66628494 160.81107576]
[  0.58558021  33.64448087 197.99245715]
[ 0.6574629  45.59676738 70.67648175]
[ 0.13748646 47.26343432 23.46686409]
[  0.76475293  92.24596085 177.48935442]
[  0.29626603  81.48909977 179.64037297]
[ 0.6236913  87.47656503 97.32992467]
[  0.46475666  46.051489   122.05853503]
[  0.57950422  29.18162993 132.20992165]
[ 0.19323371 43.90356688 60.43297631]
[1.06975546e-01 9.55782068e+01 1.43137479e+02]
[  0.14496809  13.58042645 141.96057347]
[ 0.12766379 92.64949035 84.71427892]
[ 0.35322393 58.6275147  51.28418714]
[  0.46834012  36.55862911 100.17175203]
[ 0.18550452 11.93657671 55.12922944]
[ 0.6262667  35.61675404 90.47918611]
[  0.29414947  12.63676964 118.63542406]
[  0.62048755  52.28043449 164.18981236]

# RDFWFV_20201222_V10

In [15]:
#     num_input = 4 # RDF, WFV, RDF+WFV
#     onehot = 2
#     num_output = 6
#     num_in_cycle = 50
#     num_of_cycle = [50, 50, 100]
#     num_total = [x*num_in_cycle for x in num_of_cycle]
#     header = 0

file_path = '2020_RDFWFV_20201222_V10.xlsx'

#### TRAIN

## 1.read from excel
data_x = pd.read_excel('../'+file_path, sheet_name='RDFWFV data', usecols="C:F", nrows=num_total[2]+1, header=header)
data_y = pd.read_excel('../'+file_path, sheet_name='RDFWFV data', usecols="G:N", nrows=num_total[2]+1, header=header)

## 2.one-hot encoding (나중에 1, 0 을 인위적으로 넣어주기)
X_all , Y_all = np.zeros((num_total[2], num_input)), np.zeros((num_total[2], num_output))
X_per_cycle, Y_per_cycle = np.zeros((num_of_cycle[2], num_input)), np.zeros((num_of_cycle[2], num_output))

## 3. DATA_X DATA_Y preprocessing
# Y: IDLO, IDHI
data_y = data_y.drop('Idlo', axis=1)
data_y = data_y.drop('Idhi', axis=1)

print(data_y)

## 4. X_per_cycle
for i in range(num_of_cycle[2]):
#     print(i*num_in_cycle_train, i*num_in_cycle_train+1)
    X_per_cycle[i] = data_x[i*num_in_cycle:i*num_in_cycle+1].values
#     print(X_per_cycle_train[i])

## 5. X_all
X_all = np.repeat(X_per_cycle,num_in_cycle,axis=0)


## 6. LER RDF WFV -> one-hot 
# RDFWFV 임 (11)

X_all = np.hstack((X_all, np.ones((X_all.shape[0], 1))))
X_per_cycle = np.hstack((X_per_cycle, np.ones((X_per_cycle.shape[0], 1))))

X_all = np.hstack((X_all, np.ones((X_all.shape[0], 1))))
X_per_cycle = np.hstack((X_per_cycle, np.ones((X_per_cycle.shape[0], 1))))


## 7. Y_all
for i in range(num_of_cycle[2]):
#     print(i*num_in_cycle_train,(i+1)*num_in_cycle_train)
    Y_all[i*num_in_cycle:(i+1)*num_in_cycle] = data_y[i*num_in_cycle:(i+1)*num_in_cycle].values
#     print(Y_all_train[i*num_in_cycle_train])

## 8. Y_per_cycle    
for i in range(num_of_cycle[2]):
    Y_per_cycle[i] = np.mean(Y_all[i*num_in_cycle:(i+1)*num_in_cycle],axis=0)
#     print(i, Y_per_cycle_train[i])

## 9. Y_per_mean_cov
Y_per_mean_cov = mean_cov(Y_all, num_in_cycle, num_of_cycle[2], num_output)
print(Y_per_mean_cov.shape)
for i in range(len(Y_per_mean_cov)):
    print(Y_per_mean_cov[i])

print("============ TRAIN Data load =============")
print()
print("X data shape: ", X_all.shape, "X per cycle data shape:", X_per_cycle.shape)
print("Y data shape: ", Y_all.shape, "Y per cycle data shape:", Y_per_cycle.shape)  
print("Y per mean cov shape: ", Y_per_mean_cov.shape)
print("any nan in X?: ", np.argwhere(np.isnan(X_all)))
print("any nan in Y?: ", np.argwhere(np.isnan(Y_all)))  
print()
print("============ TRAIN Data save =============")
print()
data_train = []
data_train.append(X_all)
data_train.append(Y_all)
data_train.append(X_per_cycle)
data_train.append(Y_per_cycle)
data_train.append(Y_per_mean_cov)
    
data_train = np.array(data_train)
print("saved data shape: ", data_train.shape)
np.save('./rdfwfv_train'+file_path, data_train)
print()
    

              Ioff  Vtlin  Vtsat     Idlin     Idsat      SS
0     3.570000e-11  0.285  0.277  0.000024  0.000057  63.283
1     1.651000e-11  0.316  0.293  0.000024  0.000056  62.388
2     1.117000e-11  0.338  0.325  0.000023  0.000051  65.969
3     1.916000e-12  0.379  0.354  0.000022  0.000046  63.341
4     2.597000e-12  0.357  0.347  0.000023  0.000049  63.126
...            ...    ...    ...       ...       ...     ...
4995  3.492000e-12  0.340  0.326  0.000009  0.000027  60.761
4996  2.046000e-12  0.355  0.345  0.000008  0.000022  61.140
4997  2.312000e-12  0.356  0.347  0.000008  0.000024  61.816
4998  1.211000e-11  0.310  0.302  0.000010  0.000030  61.313
4999  3.424000e-12  0.360  0.349  0.000009  0.000026  63.035

[5000 rows x 6 columns]
Y_all size:  (5000, 6)
(100, 27)
[ 2.50579760e-11  3.26700000e-01  3.12380000e-01  2.38966000e-05
  5.37716000e-05  6.39898400e+01  1.21040162e-21  1.79886735e-03
  1.79848531e-03  4.18746780e-12  5.87155035e-11  3.93802973e+00
 -1.07869424e-1

In [16]:
##### train
file_path = 'rdfwfv_train2020_RDFWFV_20201222_V10.xlsx'

data = np.load(file_path+'.npy', allow_pickle=True)

X_all, Y_all, X_per_cycle, Y_per_cycle, Y_per_mean_cov = data[0], data[1], data[2], data[3], data[4]

print(data[0].shape, data[1].shape, data[2].shape, data[3].shape, data[4].shape)

for i in range(X_per_cycle.shape[0]):
    print(X_per_cycle[i])
      
for i in range(Y_per_mean_cov.shape[0]):
    print(Y_per_mean_cov[i])
    
print("break")    

(5000, 6) (5000, 6) (100, 6) (100, 6) (100, 27)
[1.49050000e-02 7.80666160e+20 3.40063127e+18 2.11108806e-02
 1.00000000e+00 1.00000000e+00]
[1.62377213e-03 7.51948685e+19 1.37124501e+19 3.52468026e-02
 1.00000000e+00 1.00000000e+00]
[8.56005623e-03 1.87097104e+20 9.46502642e+18 3.20268907e-02
 1.00000000e+00 1.00000000e+00]
[1.63119303e-03 2.81634706e+19 1.73101344e+18 2.71307762e-02
 1.00000000e+00 1.00000000e+00]
[1.70562926e-02 2.82778528e+20 3.02371422e+18 1.53056517e-02
 1.00000000e+00 1.00000000e+00]
[1.76780428e-02 2.69024529e+20 1.69497671e+18 3.87215207e-02
 1.00000000e+00 1.00000000e+00]
[4.52506379e-03 6.32778976e+20 1.47277577e+19 1.79596611e-02
 1.00000000e+00 1.00000000e+00]
[9.57721414e-03 8.04553964e+20 5.86532631e+18 3.77374269e-02
 1.00000000e+00 1.00000000e+00]
[1.27725375e-02 5.00214443e+19 3.76491154e+18 1.67131121e-02
 1.00000000e+00 1.00000000e+00]
[8.12389552e-03 8.98548288e+20 2.05192597e+19 2.12069142e-02
 1.00000000e+00 1.00000000e+00]
[1.81900251e-03 8.5226

In [18]:
# WFV

file_path = '2020_RDFWFV_20201222_V10.xlsx'

#### TRAIN

## 1.read from excel
data_x = pd.read_excel('../'+file_path, sheet_name='WFV data', usecols="C:F", nrows=num_total[1]+1, header=header)
data_y = pd.read_excel('../'+file_path, sheet_name='WFV data', usecols="G:N", nrows=num_total[1]+1, header=header)

## 2.data frame (나중에 1, 0 을 인위적으로 넣어주기)
X_all , Y_all = np.zeros((num_total[1], num_input)), np.zeros((num_total[1], num_output))
X_per_cycle, Y_per_cycle = np.zeros((num_of_cycle[1], num_input)), np.zeros((num_of_cycle[1], num_output))

## 3. DATA_X DATA_Y preprocessing
# Y: IDLO, IDHI
data_y = data_y.drop('Idlo', axis=1)
data_y = data_y.drop('Idhi', axis=1)

print(data_y)

## 4. X_per_cycle
for i in range(num_of_cycle[1]):
#     print(i*num_in_cycle_train, i*num_in_cycle_train+1)
    X_per_cycle[i] = data_x[i*num_in_cycle:i*num_in_cycle+1].values
#     print(X_per_cycle_train[i])

## 5. X_all
X_all = np.repeat(X_per_cycle,num_in_cycle,axis=0)

## (additional) AGS switch
temp_AGS = X_per_cycle[:, -1].reshape(-1, 1)
X_per_cycle = np.delete(X_per_cycle, -1, axis=1)
X_per_cycle = np.hstack((temp_AGS, X_per_cycle))

temp_AGS = X_all[:, -1].reshape(-1, 1)
X_all = np.delete(X_all, -1, axis=1)
X_all = np.hstack((temp_AGS, X_all))

## 6. LER RDF WFV -> one-hot 
# WFV 임 (001)
X_all = np.hstack((X_all, np.zeros((X_all.shape[0], 1))))
X_per_cycle = np.hstack((X_per_cycle, np.zeros((X_per_cycle.shape[0], 1))))

X_all = np.hstack((X_all, np.ones((X_all.shape[0], 1))))
X_per_cycle = np.hstack((X_per_cycle, np.ones((X_per_cycle.shape[0], 1))))

for i in range(X_per_cycle.shape[0]):
    print(X_per_cycle[i])

## 7. Y_all
for i in range(num_of_cycle[1]):
#     print(i*num_in_cycle_train,(i+1)*num_in_cycle_train)
    Y_all[i*num_in_cycle:(i+1)*num_in_cycle] = data_y[i*num_in_cycle:(i+1)*num_in_cycle].values
#     print(Y_all_train[i*num_in_cycle_train])

## 8. Y_per_cycle    
for i in range(num_of_cycle[1]):
    Y_per_cycle[i] = np.mean(Y_all[i*num_in_cycle:(i+1)*num_in_cycle],axis=0)
#     print(i, Y_per_cycle_train[i])

## 9. Y_per_mean_cov
Y_per_mean_cov = mean_cov(Y_all, num_in_cycle, num_of_cycle[1], num_output)
print(Y_per_mean_cov.shape)
for i in range(len(Y_per_mean_cov)):
    print(Y_per_mean_cov[i])

# switch ags to left


print("============ TRAIN Data load =============")
print()
print("X data shape: ", X_all.shape, "X per cycle data shape:", X_per_cycle.shape)
print("Y data shape: ", Y_all.shape, "Y per cycle data shape:", Y_per_cycle.shape)  
print("any nan in X?: ", np.argwhere(np.isnan(X_all)))
print("any nan in Y?: ", np.argwhere(np.isnan(Y_all)))  
print()
print("============ TRAIN Data save =============")
print()
data_train = []
data_train.append(X_all)
data_train.append(Y_all)
data_train.append(X_per_cycle)
data_train.append(Y_per_cycle)
data_train.append(Y_per_mean_cov)

data_train = np.array(data_train)
print("saved data shape: ", data_train.shape)
np.save('./wfv_train'+file_path, data_train)
print()

              Ioff  Vtlin  Vtsat     Idlin     Idsat      SS
0     2.703000e-11  0.290  0.283  0.000032  0.000059  62.901
1     1.186000e-11  0.321  0.299  0.000032  0.000061  62.270
2     3.794000e-12  0.356  0.350  0.000030  0.000052  64.063
3     1.247000e-12  0.386  0.361  0.000028  0.000050  62.222
4     1.319000e-12  0.373  0.363  0.000029  0.000047  62.514
...            ...    ...    ...       ...       ...     ...
2496  3.999000e-12  0.337  0.333  0.000027  0.000051  62.634
2497  4.579000e-12  0.332  0.322  0.000028  0.000057  62.099
2498  3.646000e-12  0.339  0.331  0.000027  0.000053  62.261
2499  3.880000e-12  0.339  0.328  0.000027  0.000055  61.992
2500           NaN    NaN    NaN       NaN       NaN     NaN

[2501 rows x 6 columns]
[1.42908567e-02 9.77282068e+20 1.40362085e+20 1.55974902e-02
 0.00000000e+00 1.00000000e+00]
[1.48403238e-02 2.32417116e+20 2.03172950e+19 3.25928432e-02
 0.00000000e+00 1.00000000e+00]
[5.64483335e-03 2.19437191e+20 1.86124046e+19 3.32187120e

In [21]:
##### train
file_path = 'wfv_train2020_RDFWFV_20201222_V10.xlsx'

data = np.load(file_path+'.npy', allow_pickle=True)

X_all, Y_all, X_per_cycle, Y_per_cycle, Y_per_mean_cov = data[0], data[1], data[2], data[3], data[4]

print(data[0].shape, data[1].shape, data[2].shape, data[3].shape, data[4].shape)

for i in range(X_per_cycle.shape[0]):
    print(X_per_cycle[i])

for i in range(Y_all.shape[0]):
    print(Y_all[i])

print("break")       
    

(2500, 6) (2500, 6) (50, 6) (50, 6) (50, 27)
[1.42908567e-02 9.77282068e+20 1.40362085e+20 1.55974902e-02
 0.00000000e+00 1.00000000e+00]
[1.48403238e-02 2.32417116e+20 2.03172950e+19 3.25928432e-02
 0.00000000e+00 1.00000000e+00]
[5.64483335e-03 2.19437191e+20 1.86124046e+19 3.32187120e-02
 0.00000000e+00 1.00000000e+00]
[1.74036053e-02 4.18849334e+20 2.89901264e+19 2.13256782e-02
 0.00000000e+00 1.00000000e+00]
[2.45406918e-03 5.85955460e+19 8.07154645e+18 3.04992697e-02
 0.00000000e+00 1.00000000e+00]
[7.82787312e-03 3.65572154e+19 3.39932085e+18 2.21151442e-02
 0.00000000e+00 1.00000000e+00]
[1.54502717e-02 2.12510338e+19 1.08637683e+18 3.91987742e-02
 0.00000000e+00 1.00000000e+00]
[1.66838976e-02 1.23296290e+20 1.99314089e+19 1.56752964e-02
 0.00000000e+00 1.00000000e+00]
[1.01675853e-02 4.47202618e+19 5.63399659e+18 1.71093627e-02
 0.00000000e+00 1.00000000e+00]
[1.99652583e-02 3.58444577e+19 3.76715224e+18 3.20694046e-02
 0.00000000e+00 1.00000000e+00]
[1.32853303e-02 8.7735153

[4.6450e-12 3.4600e-01 3.2900e-01 2.8110e-05 5.3700e-05 6.2743e+01]
[4.3350e-12 3.4500e-01 3.3500e-01 2.8410e-05 5.2770e-05 6.3905e+01]
[8.1300e-12 3.2700e-01 3.1200e-01 3.0230e-05 5.9940e-05 6.3692e+01]
[3.3610e-12 3.4400e-01 3.3400e-01 2.8410e-05 5.3850e-05 6.2447e+01]
[7.265e-12 3.290e-01 3.210e-01 2.899e-05 5.428e-05 6.285e+01]
[3.6250e-12 3.5900e-01 3.3600e-01 2.8060e-05 5.2180e-05 6.3312e+01]
[1.6860e-11 3.1600e-01 2.9400e-01 2.9650e-05 5.8850e-05 6.2795e+01]
[2.8920e-12 3.5000e-01 3.4500e-01 2.9760e-05 5.4350e-05 6.3955e+01]
[4.1080e-12 3.4000e-01 3.3500e-01 2.9010e-05 5.5200e-05 6.3176e+01]
[1.670e-11 3.030e-01 2.900e-01 3.153e-05 6.501e-05 6.270e+01]
[4.0340e-12 3.3700e-01 3.2800e-01 2.9680e-05 5.8070e-05 6.2397e+01]
[1.1160e-11 3.2100e-01 3.0700e-01 2.9120e-05 5.8330e-05 6.3796e+01]
[8.4380e-12 3.2800e-01 3.1600e-01 2.8820e-05 5.5640e-05 6.3199e+01]
[5.5040e-12 3.3300e-01 3.2500e-01 2.9470e-05 5.6170e-05 6.3084e+01]
[5.9320e-12 3.3300e-01 3.2200e-01 2.8940e-05 5.6060e-05 6.28

[2.3640e-13 4.1300e-01 4.0500e-01 2.4440e-05 4.0400e-05 6.2183e+01]
[1.3340e-11 3.0800e-01 3.0100e-01 2.7610e-05 5.3900e-05 6.2731e+01]
[4.7950e-11 2.7800e-01 2.6700e-01 2.8870e-05 5.5260e-05 6.2717e+01]
[2.3030e-13 4.1700e-01 4.0200e-01 2.3920e-05 4.0600e-05 6.1827e+01]
[7.9000e-13 3.8000e-01 3.7400e-01 2.5450e-05 4.4990e-05 6.2388e+01]
[2.5440e-11 3.1400e-01 2.7900e-01 2.7610e-05 5.6060e-05 6.2249e+01]
[1.0220e-12 3.9600e-01 3.6500e-01 2.3900e-05 4.2940e-05 6.1851e+01]
[4.0630e-12 3.4200e-01 3.3000e-01 2.8580e-05 5.4450e-05 6.2689e+01]
[6.7650e-12 3.4000e-01 3.1100e-01 2.7610e-05 5.5110e-05 6.1884e+01]
[1.2400e-11 3.1000e-01 3.0600e-01 2.8410e-05 5.1570e-05 6.2919e+01]
[2.1070e-12 3.5400e-01 3.4600e-01 2.6250e-05 4.8260e-05 6.1995e+01]
[6.491e-12 3.320e-01 3.230e-01 2.676e-05 4.732e-05 6.250e+01]
[1.8650e-12 3.6000e-01 3.5600e-01 2.7060e-05 4.5370e-05 6.2449e+01]
[6.7430e-11 2.7600e-01 2.5300e-01 3.0050e-05 6.3690e-05 6.2607e+01]
[6.9700e-12 3.2900e-01 3.2700e-01 2.6730e-05 4.8650e-0

In [25]:
# RDF

file_path = '2020_RDFWFV_20201222_V10.xlsx'

#### TRAIN

## 1.read from excel
data_x = pd.read_excel('../'+file_path, sheet_name='RDF data', usecols="C:E", nrows=num_total[0]+1, header=header)
data_y = pd.read_excel('../'+file_path, sheet_name='RDF data', usecols="F:M", nrows=num_total[0]+1, header=header)

## 2.data frame (나중에 1, 0 을 인위적으로 넣어주기)
X_all , Y_all = np.zeros((num_total[0], num_input-1)), np.zeros((num_total[0], num_output))
X_per_cycle, Y_per_cycle = np.zeros((num_of_cycle[0], num_input-1)), np.zeros((num_of_cycle[0], num_output))

## 3. DATA_X DATA_Y preprocessing
# Y: IDLO, IDHI
data_y = data_y.drop('Idlo', axis=1)
data_y = data_y.drop('Idhi', axis=1)

print(data_y)

## 4. X_per_cycle
for i in range(num_of_cycle[0]):
#     print(i*num_in_cycle_train, i*num_in_cycle_train+1)
    X_per_cycle[i] = data_x[i*num_in_cycle:i*num_in_cycle+1].values
#     print(X_per_cycle_train[i])

# add AGS to first column
X_per_cycle = np.hstack((np.zeros((X_per_cycle.shape[0],1)), X_per_cycle))


## 5. X_all
X_all = np.repeat(X_per_cycle,num_in_cycle,axis=0)


## 6. LER RDF WFV -> one-hot 
# RDF 임 (10)

X_all = np.hstack((X_all, np.ones((X_all.shape[0], 1))))
X_per_cycle = np.hstack((X_per_cycle, np.ones((X_per_cycle.shape[0], 1))))

X_all = np.hstack((X_all, np.zeros((X_all.shape[0], 1))))
X_per_cycle = np.hstack((X_per_cycle, np.zeros((X_per_cycle.shape[0], 1))))


## 7. Y_all
for i in range(num_of_cycle[0]):
#     print(i*num_in_cycle_train,(i+1)*num_in_cycle_train)
    Y_all[i*num_in_cycle:(i+1)*num_in_cycle] = data_y[i*num_in_cycle:(i+1)*num_in_cycle].values
#     print(Y_all_train[i*num_in_cycle_train])

## 8. Y_per_cycle    
for i in range(num_of_cycle[0]):
    Y_per_cycle[i] = np.mean(Y_all[i*num_in_cycle:(i+1)*num_in_cycle],axis=0)
#     print(i, Y_per_cycle_train[i])

## 9. Y_per_mean_cov
Y_per_mean_cov = mean_cov(Y_all, num_in_cycle, num_of_cycle[0], num_output)
print(Y_per_mean_cov.shape)
for i in range(len(Y_per_mean_cov)):
    print(Y_per_mean_cov[i])

print("============ TRAIN Data load =============")
print()
print("X data shape: ", X_all.shape, "X per cycle data shape:", X_per_cycle.shape)
print("Y data shape: ", Y_all.shape, "Y per cycle data shape:", Y_per_cycle.shape)  
print("any nan in X?: ", np.argwhere(np.isnan(X_all)))
print("any nan in Y?: ", np.argwhere(np.isnan(Y_all)))  
print()
print("============ TRAIN Data save =============")
print()
data_train = []
data_train.append(X_all)
data_train.append(Y_all)
data_train.append(X_per_cycle)
data_train.append(Y_per_cycle)
data_train.append(Y_per_mean_cov)
    
data_train = np.array(data_train)
print("saved data shape: ", data_train.shape)
np.save('./rdf_train'+file_path, data_train)
print()

# #### TEST

# ## 1.read from excel
# test_data_x = pd.read_excel('../'+file_path, sheet_name='RDF testdata', usecols="B:D", nrows=num_total_test[0]+1, header=header)
# test_data_y = pd.read_excel('../'+file_path, sheet_name='RDF testdata', usecols="E:L", nrows=num_total_test[0]+1, header=header)

# ## 2.one-hot encoding (나중에 1, 0 을 인위적으로 넣어주기)
# X_all_test , Y_all_test = np.zeros((num_total_test[0], num_input[0]-3)), np.zeros((num_total_test[0], num_output))
# X_per_cycle_test, Y_per_cycle_test = np.zeros((num_of_cycle_test[0], num_input[0]-3)), np.zeros((num_of_cycle_test[0], num_output))

# ## 3. DATA_X DATA_Y preprocessing
# test_data_y = test_data_y.drop('Idlo', axis=1)
# test_data_y = test_data_y.drop('Idhi', axis=1)

# ## 4. X_per_cycle, Y_all
# for i in range(num_of_cycle_test[0]):
#     X_per_cycle_test[i] = test_data_x[i*num_in_cycle_test:i*num_in_cycle_test+1].values
    
# Y_all_test = test_data_y.values    

# ## 5. X_all, Y_per_cycle
# for i in range(num_of_cycle_test[0]):
# #     print(i*num_in_cycle_test, (i+1)*num_in_cycle_test)
# #     print(X_per_cycle_test[i])
# #     print(np.reshape(X_per_cycle_test[i], (1,3)).shape)
#     X_all_test[i*num_in_cycle_test:(i+1)*num_in_cycle_test] = np.repeat(np.reshape(X_per_cycle_test[i], (1,3)), num_in_cycle_test, axis=0)
# #     print(X_all_test[i*num_in_cycle_test:(i+1)*num_in_cycle_test])

#     Y_per_cycle_test[i] = np.mean(Y_all_test[i*num_in_cycle_test:(i+1)*num_in_cycle_test], axis=0)

# ## 6. 인위적으로 one-hot 넣어주기
# X_all_test = np.hstack((X_all_test, np.zeros((X_all_test.shape[0], 1))))
# X_per_cycle_test = np.hstack((X_per_cycle_test, np.zeros((X_per_cycle_test.shape[0], 1))))    

# X_all_test = np.hstack((X_all_test, np.ones((X_all_test.shape[0], 1))))
# X_per_cycle_test = np.hstack((X_per_cycle_test, np.ones((X_per_cycle_test.shape[0], 1))))

# X_all_test = np.hstack((X_all_test, np.zeros((X_all_test.shape[0], 1))))
# X_per_cycle_test = np.hstack((X_per_cycle_test, np.zeros((X_per_cycle_test.shape[0], 1))))
    
# ## additionally add Average Grain size
# X_all_test = np.hstack((np.zeros((X_all_test.shape[0], 1)), X_all_test))
# X_per_cycle_test = np.hstack((np.zeros((X_per_cycle_test.shape[0], 1)), X_per_cycle_test))
# # for i in range(X_all_test.shape[0]):
# #     print(X_all_test[i])
# #     print(Y_all_test[i])
    
    
# print("============ TEST Data load =============")
# print()
# print("X data shape: ", X_all_test.shape, "X per cycle data shape:", X_per_cycle_test.shape)
# print("Y data shape: ", Y_all_test.shape, "Y per cycle data shape:", Y_per_cycle_test.shape)  
# print("any nan in X?: ", np.argwhere(np.isnan(X_all_test)))
# print("any nan in Y?: ", np.argwhere(np.isnan(Y_all_test)))  
# print()
# print("============ TEST Data save =============")
# print()
# data_test = []
# data_test.append(X_all_test)
# data_test.append(Y_all_test)
# data_test.append(X_per_cycle_test)
# data_test.append(Y_per_cycle_test)
    
# data_test = np.array(data_test)
# print("saved data shape: ", data_test.shape)
# np.save('./rdf_test'+file_path, data_test)
# print()


# # # RDF+WFV  
    

              Ioff  Vtlin  Vtsat     Idlin     Idsat      SS
0     2.161000e-12  0.348  0.341  0.000007  0.000021  61.024
1     2.116000e-12  0.348  0.342  0.000008  0.000023  61.121
2     2.125000e-12  0.348  0.342  0.000008  0.000022  61.073
3     2.318000e-12  0.346  0.340  0.000008  0.000026  61.110
4     2.172000e-12  0.347  0.341  0.000008  0.000022  60.965
...            ...    ...    ...       ...       ...     ...
2495  2.367000e-12  0.346  0.339  0.000010  0.000028  61.366
2496  2.628000e-12  0.344  0.337  0.000010  0.000032  61.340
2497  2.442000e-12  0.345  0.338  0.000010  0.000030  61.289
2498  2.532000e-12  0.345  0.337  0.000010  0.000032  61.319
2499  2.365000e-12  0.346  0.340  0.000010  0.000030  61.415

[2500 rows x 6 columns]
Y_all size:  (2500, 6)
(50, 27)
[ 2.12202000e-12  3.48380000e-01  3.41900000e-01  7.52316000e-06
  2.17530000e-05  6.10259600e+01  1.53623057e-26  1.50571429e-06
  2.62244898e-06  5.87989943e-14  2.60040510e-12  2.50028408e-03
 -1.36558776e-16

In [26]:
##### train
file_path = 'rdf_train2020_RDFWFV_20201222_V10.xlsx'

data = np.load(file_path+'.npy', allow_pickle=True)

X_all, Y_all, X_per_cycle, Y_per_cycle, Y_per_mean_cov = data[0], data[1], data[2], data[3], data[4]

print(data[0].shape, data[1].shape, data[2].shape, data[3].shape, data[4].shape)

for i in range(X_per_cycle.shape[0]):
    print(X_per_cycle[i])

for i in range(Y_all.shape[0]):
    print(Y_all[i])

# for i in range(Y_per_mean_cov.shape[0]):
#     print(i)
#     print(Y_per_mean_cov[i])
    
print("break")    

(2500, 6) (2500, 6) (50, 6) (50, 6) (50, 27)
[0.00000000e+00 2.41711626e+19 2.60848106e+18 1.95521349e-02
 1.00000000e+00 0.00000000e+00]
[0.00000000e+00 6.00746645e+19 5.90684912e+18 2.48157453e-02
 1.00000000e+00 0.00000000e+00]
[0.00000000e+00 4.91906800e+19 8.21088103e+18 2.88600593e-02
 1.00000000e+00 0.00000000e+00]
[0.00000000e+00 6.93972773e+20 1.19787315e+20 3.25943890e-02
 1.00000000e+00 0.00000000e+00]
[0.00000000e+00 1.97147165e+20 3.35404987e+19 3.08166199e-02
 1.00000000e+00 0.00000000e+00]
[0.00000000e+00 5.61023517e+19 6.25309331e+18 1.41064160e-02
 1.00000000e+00 0.00000000e+00]
[0.00000000e+00 1.89735925e+20 2.42511816e+19 3.72088631e-02
 1.00000000e+00 0.00000000e+00]
[0.00000000e+00 4.84929301e+20 3.89096050e+19 2.13662976e-02
 1.00000000e+00 0.00000000e+00]
[0.00000000e+00 5.41926737e+19 4.46384917e+18 2.58886663e-02
 1.00000000e+00 0.00000000e+00]
[0.00000000e+00 4.06227776e+19 6.45178569e+18 3.96584500e-02
 1.00000000e+00 0.00000000e+00]
[0.00000000e+00 4.7749628

[2.6560e-12 3.4400e-01 3.3600e-01 1.4400e-05 3.9420e-05 6.1461e+01]
[2.7790e-12 3.4400e-01 3.3600e-01 1.4520e-05 4.0490e-05 6.1517e+01]
[2.6100e-12 3.4400e-01 3.3700e-01 1.5090e-05 4.0360e-05 6.1571e+01]
[2.7150e-12 3.4400e-01 3.3600e-01 1.4110e-05 3.8590e-05 6.1455e+01]
[2.8470e-12 3.4200e-01 3.3500e-01 1.3540e-05 3.8300e-05 6.1447e+01]
[2.5310e-12 3.4500e-01 3.3700e-01 1.2500e-05 3.5710e-05 6.1375e+01]
[2.6590e-12 3.4300e-01 3.3700e-01 1.3210e-05 3.6080e-05 6.1446e+01]
[2.7960e-12 3.4300e-01 3.3600e-01 1.3880e-05 3.8730e-05 6.1514e+01]
[2.7320e-12 3.4300e-01 3.3600e-01 1.3560e-05 3.8590e-05 6.1422e+01]
[2.6030e-12 3.4400e-01 3.3700e-01 1.2730e-05 3.6000e-05 6.1368e+01]
[2.7760e-12 3.4300e-01 3.3500e-01 1.2670e-05 3.6350e-05 6.1439e+01]
[2.6440e-12 3.4400e-01 3.3700e-01 1.3300e-05 3.6590e-05 6.1472e+01]
[2.8290e-12 3.4300e-01 3.3500e-01 1.3340e-05 3.8140e-05 6.1395e+01]
[2.5210e-12 3.4500e-01 3.3800e-01 1.3770e-05 3.6440e-05 6.1396e+01]
[2.6190e-12 3.4500e-01 3.3700e-01 1.3540e-05 3.6

[3.5030e-12 3.4000e-01 3.3100e-01 1.7820e-05 4.5810e-05 6.1872e+01]
[3.348e-12 3.410e-01 3.320e-01 1.760e-05 4.571e-05 6.180e+01]
[3.4320e-12 3.4000e-01 3.3100e-01 1.9170e-05 4.6990e-05 6.1914e+01]
[3.463e-12 3.400e-01 3.310e-01 1.834e-05 4.740e-05 6.182e+01]
[3.3520e-12 3.4100e-01 3.3200e-01 1.8240e-05 4.5760e-05 6.1809e+01]
[3.340e-12 3.410e-01 3.320e-01 1.790e-05 4.626e-05 6.181e+01]
[3.3290e-12 3.4100e-01 3.3200e-01 1.8240e-05 4.6190e-05 6.1837e+01]
[3.481e-12 3.390e-01 3.310e-01 1.827e-05 4.687e-05 6.193e+01]
[3.2650e-12 3.4200e-01 3.3200e-01 1.7530e-05 4.5630e-05 6.1803e+01]
[3.3790e-12 3.4000e-01 3.3200e-01 1.7880e-05 4.6280e-05 6.1838e+01]
[3.2230e-12 3.4100e-01 3.3300e-01 1.8100e-05 4.6200e-05 6.1769e+01]
[3.3520e-12 3.4000e-01 3.3200e-01 1.8720e-05 4.6070e-05 6.1932e+01]
[3.8350e-12 3.3800e-01 3.2900e-01 2.0180e-05 5.0210e-05 6.1937e+01]
[2.8990e-12 3.4300e-01 3.3500e-01 1.7600e-05 4.3620e-05 6.1715e+01]
[3.2020e-12 3.4100e-01 3.3200e-01 1.7880e-05 4.5070e-05 6.1744e+01]
[3.2

[1.9140e-12 3.5100e-01 3.4400e-01 6.4190e-06 1.8030e-05 6.0904e+01]
[1.9010e-12 3.5200e-01 3.4500e-01 6.0800e-06 1.6500e-05 6.0937e+01]
[1.913e-12 3.510e-01 3.450e-01 6.358e-06 1.708e-05 6.089e+01]
[1.9910e-12 3.4900e-01 3.4300e-01 6.4050e-06 1.8070e-05 6.0896e+01]
[1.8580e-12 3.5200e-01 3.4600e-01 6.2390e-06 1.6770e-05 6.0925e+01]
[1.8860e-12 3.5100e-01 3.4500e-01 5.9830e-06 1.6610e-05 6.0898e+01]
[2.2660e-12 3.4700e-01 3.4100e-01 6.1990e-06 1.7620e-05 6.1018e+01]
[1.8730e-12 3.5200e-01 3.4600e-01 6.1680e-06 1.6280e-05 6.0964e+01]
[1.9800e-12 3.5100e-01 3.4400e-01 6.1660e-06 1.7590e-05 6.0863e+01]
[1.9290e-12 3.5000e-01 3.4400e-01 6.4930e-06 1.8040e-05 6.0861e+01]
[1.7280e-12 3.5400e-01 3.4600e-01 6.1630e-06 1.8800e-05 6.0848e+01]
[1.6880e-12 3.5500e-01 3.4800e-01 5.9560e-06 1.4070e-05 6.0903e+01]
[1.8600e-12 3.5200e-01 3.4500e-01 6.1610e-06 1.7760e-05 6.0866e+01]
[1.9340e-12 3.5100e-01 3.4400e-01 6.5540e-06 1.8930e-05 6.0903e+01]
[1.9640e-12 3.5000e-01 3.4400e-01 6.0220e-06 1.8440e-0

In [27]:
# WFVRDF + WFV + RDF

file_path = '2020_RDFWFV_20201222_V10.xlsx'

#### TRAIN

rdfwfv_data = np.load('rdfwfv_train' + file_path+'.npy', allow_pickle=True)
wfv_data = np.load('wfv_train'+file_path+'.npy', allow_pickle=True)
rdf_data = np.load('rdf_train'+file_path+'.npy', allow_pickle=True)

print(rdfwfv_data[0].shape, rdfwfv_data[1].shape, rdfwfv_data[2].shape, rdfwfv_data[3].shape, rdfwfv_data[4].shape)
print(wfv_data[0].shape, wfv_data[1].shape, wfv_data[2].shape, wfv_data[3].shape, wfv_data[4].shape)
print(rdf_data[0].shape, rdf_data[1].shape, rdf_data[2].shape, rdf_data[3].shape, rdf_data[4].shape)

total_X_all = np.concatenate((rdfwfv_data[0], wfv_data[0], rdf_data[0]), axis=0)
total_Y_all = np.concatenate((rdfwfv_data[1], wfv_data[1], rdf_data[1]), axis=0)
total_X_per_cycle = np.concatenate((rdfwfv_data[2], wfv_data[2], rdf_data[2]), axis=0)
total_Y_per_cycle = np.concatenate((rdfwfv_data[3], wfv_data[3], rdf_data[3]), axis=0)
total_Y_per_mean_cov = np.concatenate((rdfwfv_data[4], wfv_data[4], rdf_data[4]), axis=0)
print(total_X_all.shape, total_Y_all.shape, total_X_per_cycle.shape, total_Y_per_cycle.shape, total_Y_per_mean_cov.shape)


print("============ TRAIN Data load =============")
print()
print("X data shape: ", total_X_all.shape, "X per cycle data shape:", total_X_per_cycle.shape)
print("Y data shape: ", total_Y_all.shape, "Y per cycle data shape:", total_Y_per_cycle.shape)  
print("any nan in X?: ", np.argwhere(np.isnan(total_X_all)))
print("any nan in Y?: ", np.argwhere(np.isnan(total_Y_all)))
print()
print("============ TRAIN Data save =============")
print()
data_train = []
data_train.append(total_X_all)
data_train.append(total_Y_all)
data_train.append(total_X_per_cycle)
data_train.append(total_Y_per_cycle)
data_train.append(total_Y_per_mean_cov)
    
data_train = np.array(data_train)
print("saved data shape: ", data_train.shape)
np.save('./rdfwfv_wfv_rdf_train'+file_path, data_train)
print()

(5000, 6) (5000, 6) (100, 6) (100, 6) (100, 27)
(2500, 6) (2500, 6) (50, 6) (50, 6) (50, 27)
(2500, 6) (2500, 6) (50, 6) (50, 6) (50, 27)
(10000, 6) (10000, 6) (200, 6) (200, 6) (200, 27)
============ TRAIN Data load =============

X data shape:  (10000, 6) X per cycle data shape: (200, 6)
Y data shape:  (10000, 6) Y per cycle data shape: (200, 6)
any nan in X?:  []
any nan in Y?:  []

============ TRAIN Data save =============

saved data shape:  (5,)



In [29]:
##### train
file_path = 'rdfwfv_wfv_rdf_train2020_RDFWFV_20201222_V10.xlsx'

data = np.load(file_path+'.npy', allow_pickle=True)

X_all, Y_all, X_per_cycle, Y_per_cycle, Y_per_mean_cov = data[0], data[1], data[2], data[3], data[4]

print(data[0].shape, data[1].shape, data[2].shape, data[3].shape, data[4].shape)

for i in range(X_per_cycle.shape[0]):
    print(X_per_cycle[i])

for i in range(Y_all.shape[0]):
    print(Y_all[i])

print("break")    

(10000, 6) (10000, 6) (200, 6) (200, 6) (200, 27)
[1.49050000e-02 7.80666160e+20 3.40063127e+18 2.11108806e-02
 1.00000000e+00 1.00000000e+00]
[1.62377213e-03 7.51948685e+19 1.37124501e+19 3.52468026e-02
 1.00000000e+00 1.00000000e+00]
[8.56005623e-03 1.87097104e+20 9.46502642e+18 3.20268907e-02
 1.00000000e+00 1.00000000e+00]
[1.63119303e-03 2.81634706e+19 1.73101344e+18 2.71307762e-02
 1.00000000e+00 1.00000000e+00]
[1.70562926e-02 2.82778528e+20 3.02371422e+18 1.53056517e-02
 1.00000000e+00 1.00000000e+00]
[1.76780428e-02 2.69024529e+20 1.69497671e+18 3.87215207e-02
 1.00000000e+00 1.00000000e+00]
[4.52506379e-03 6.32778976e+20 1.47277577e+19 1.79596611e-02
 1.00000000e+00 1.00000000e+00]
[9.57721414e-03 8.04553964e+20 5.86532631e+18 3.77374269e-02
 1.00000000e+00 1.00000000e+00]
[1.27725375e-02 5.00214443e+19 3.76491154e+18 1.67131121e-02
 1.00000000e+00 1.00000000e+00]
[8.12389552e-03 8.98548288e+20 2.05192597e+19 2.12069142e-02
 1.00000000e+00 1.00000000e+00]
[1.81900251e-03 8.52

[3.8490e-12 3.6600e-01 3.3700e-01 1.5370e-05 3.8890e-05 6.3339e+01]
[5.6760e-13 3.9900e-01 3.9100e-01 1.4500e-05 3.4010e-05 6.4617e+01]
[2.444e-12 3.470e-01 3.390e-01 8.352e-06 2.420e-05 6.114e+01]
[1.8460e-12 3.5500e-01 3.4400e-01 8.9850e-06 2.7620e-05 6.1012e+01]
[3.8770e-12 3.3700e-01 3.3100e-01 8.3050e-06 2.3480e-05 6.1297e+01]
[2.0060e-12 3.5500e-01 3.4500e-01 8.7440e-06 2.6720e-05 6.1148e+01]
[1.4190e-12 3.6300e-01 3.5400e-01 8.4940e-06 2.2570e-05 6.1053e+01]
[2.442e-12 3.480e-01 3.420e-01 8.548e-06 2.267e-05 6.133e+01]
[3.6430e-12 3.4500e-01 3.3200e-01 8.6110e-06 2.6130e-05 6.1258e+01]
[3.7460e-12 3.3600e-01 3.2600e-01 8.2910e-06 2.4110e-05 6.0864e+01]
[6.7820e-12 3.2900e-01 3.1300e-01 8.8160e-06 2.4950e-05 6.1161e+01]
[5.3450e-12 3.3200e-01 3.2600e-01 8.2940e-06 2.4760e-05 6.1833e+01]
[2.138e-12 3.490e-01 3.450e-01 9.088e-06 2.600e-05 6.149e+01]
[2.745e-12 3.420e-01 3.350e-01 8.752e-06 2.507e-05 6.128e+01]
[1.0500e-12 3.6700e-01 3.6400e-01 8.2570e-06 2.2880e-05 6.1274e+01]
[3.4

[1.6070e-12 3.6300e-01 3.5300e-01 1.4750e-05 3.8950e-05 6.2122e+01]
[3.1560e-12 3.4300e-01 3.3500e-01 1.4960e-05 4.0690e-05 6.2097e+01]
[3.7320e-12 3.4200e-01 3.2900e-01 1.4960e-05 4.0740e-05 6.1827e+01]
[1.1550e-11 3.1400e-01 2.9900e-01 1.6060e-05 4.5160e-05 6.1726e+01]
[6.015e-12 3.330e-01 3.140e-01 1.661e-05 4.415e-05 6.148e+01]
[8.6640e-12 3.2500e-01 3.1500e-01 1.5040e-05 4.2340e-05 6.2565e+01]
[2.3870e-12 3.5100e-01 3.4600e-01 1.5130e-05 3.9720e-05 6.2245e+01]
[5.0470e-12 3.3000e-01 3.2400e-01 1.5520e-05 4.2640e-05 6.2223e+01]
[1.9790e-12 3.5200e-01 3.4700e-01 1.5090e-05 3.8340e-05 6.1681e+01]
[5.0940e-12 3.4200e-01 3.2000e-01 1.4510e-05 4.0400e-05 6.1646e+01]
[1.0560e-11 3.1000e-01 3.0200e-01 1.6190e-05 4.4640e-05 6.1792e+01]
[9.8520e-13 3.7100e-01 3.6400e-01 1.4880e-05 3.6790e-05 6.1492e+01]
[1.6050e-12 3.5800e-01 3.5400e-01 1.5040e-05 3.8770e-05 6.1948e+01]
[2.3860e-12 3.6400e-01 3.5600e-01 1.5270e-05 3.7730e-05 6.3978e+01]
[5.3930e-12 3.3600e-01 3.2300e-01 1.4910e-05 4.1490e-0

[1.1020e-12 3.8500e-01 3.6500e-01 2.2170e-05 4.4920e-05 6.2221e+01]
[8.2510e-11 2.6500e-01 2.5600e-01 2.4120e-05 5.6360e-05 6.3466e+01]
[3.2630e-13 4.0800e-01 4.0000e-01 2.0600e-05 3.9560e-05 6.2411e+01]
[1.0450e-10 2.5500e-01 2.4300e-01 2.5580e-05 6.5550e-05 6.2662e+01]
[1.0400e-10 2.5900e-01 2.4400e-01 2.4610e-05 6.0670e-05 6.2791e+01]
[2.3820e-13 4.1600e-01 4.0500e-01 1.9500e-05 3.6480e-05 6.2107e+01]
[7.261e-13 3.920e-01 3.860e-01 2.225e-05 4.335e-05 6.356e+01]
[5.8090e-11 2.8500e-01 2.5800e-01 2.3060e-05 5.6720e-05 6.2271e+01]
[3.940e-11 2.910e-01 2.700e-01 2.096e-05 4.887e-05 6.208e+01]
[2.0460e-12 3.6400e-01 3.4900e-01 2.2700e-05 4.6220e-05 6.2361e+01]
[3.313e-11 2.940e-01 2.760e-01 2.407e-05 5.691e-05 6.274e+01]
[6.9400e-11 2.6900e-01 2.6400e-01 2.3800e-05 5.3960e-05 6.3623e+01]
[3.6910e-13 4.0600e-01 3.9200e-01 1.9890e-05 4.0020e-05 6.2098e+01]
[1.5760e-11 3.1800e-01 2.9900e-01 2.2830e-05 4.8160e-05 6.2646e+01]
[7.6050e-11 2.6400e-01 2.5500e-01 2.3420e-05 5.3090e-05 6.2735e+01

[2.8300e-12 3.5300e-01 3.4400e-01 1.9680e-05 4.3730e-05 6.2553e+01]
[7.3120e-12 3.2600e-01 3.1100e-01 2.0710e-05 5.0070e-05 6.2281e+01]
[3.5620e-12 3.4300e-01 3.3500e-01 1.8710e-05 4.3350e-05 6.2362e+01]
[2.666e-11 3.010e-01 2.870e-01 1.814e-05 4.516e-05 6.249e+01]
[2.0250e-12 3.6800e-01 3.5400e-01 1.7440e-05 4.1330e-05 6.3671e+01]
[2.4420e-11 3.0700e-01 2.8500e-01 1.9940e-05 4.9830e-05 6.2281e+01]
[2.5390e-12 3.5000e-01 3.4400e-01 2.0020e-05 4.6560e-05 6.2551e+01]
[5.6910e-12 3.3000e-01 3.2400e-01 1.9030e-05 4.8120e-05 6.2519e+01]
[7.6380e-12 3.2200e-01 3.1300e-01 1.9370e-05 4.8090e-05 6.2423e+01]
[8.1530e-12 3.2500e-01 3.1000e-01 1.9480e-05 4.9530e-05 6.2015e+01]
[5.9090e-12 3.3600e-01 3.2100e-01 1.8050e-05 4.4090e-05 6.2443e+01]
[7.7730e-12 3.3600e-01 3.1500e-01 1.7480e-05 4.3960e-05 6.2495e+01]
[5.6490e-12 3.3600e-01 3.3000e-01 1.8420e-05 4.4870e-05 6.3329e+01]
[2.7350e-12 3.5000e-01 3.3800e-01 1.8510e-05 4.4210e-05 6.1928e+01]
[1.2030e-11 3.1600e-01 3.0300e-01 1.8040e-05 4.6130e-0

...

[1.2410e-11 3.2900e-01 3.0100e-01 2.8190e-05 5.4630e-05 6.2389e+01]
[5.8950e-13 3.9600e-01 3.7800e-01 2.5870e-05 4.5540e-05 6.2172e+01]
[3.8640e-12 3.4400e-01 3.2900e-01 2.9910e-05 5.7680e-05 6.2344e+01]
[8.9690e-12 3.3000e-01 3.0300e-01 3.0420e-05 6.0520e-05 6.1962e+01]
[2.8680e-11 2.9100e-01 2.8500e-01 2.9600e-05 5.4880e-05 6.3048e+01]
[2.1090e-12 3.5500e-01 3.4500e-01 2.7960e-05 5.1830e-05 6.2362e+01]
[1.6920e-11 3.1100e-01 3.0400e-01 2.9010e-05 5.4310e-05 6.3113e+01]
[1.3680e-12 3.6700e-01 3.6300e-01 2.8420e-05 4.7330e-05 6.2516e+01]
[3.639e-11 2.910e-01 2.710e-01 3.081e-05 6.327e-05 6.246e+01]
[1.0730e-11 3.1900e-01 3.1100e-01 2.9110e-05 5.5270e-05 6.3138e+01]
[9.3010e-12 3.2300e-01 3.0600e-01 3.0760e-05 5.9430e-05 6.2116e+01]
[7.0110e-12 3.4000e-01 3.2000e-01 2.7980e-05 5.4350e-05 6.2652e+01]
[6.6210e-12 3.3900e-01 3.1700e-01 2.8160e-05 5.3610e-05 6.2677e+01]
[3.6550e-12 3.4400e-01 3.3500e-01 2.9440e-05 5.3180e-05 6.2854e+01]
[4.1890e-12 3.3600e-01 3.2600e-01 2.9080e-05 5.6140e-0

[1.629e-12 3.700e-01 3.670e-01 1.110e-05 2.716e-05 6.220e+01]
[2.8630e-12 3.4900e-01 3.3400e-01 1.1510e-05 3.0880e-05 6.1504e+01]
[1.9090e-12 3.6000e-01 3.5200e-01 1.1160e-05 2.6820e-05 6.1439e+01]
[5.7220e-12 3.3300e-01 3.2700e-01 1.1360e-05 2.9340e-05 6.2044e+01]
[1.2420e-12 3.6900e-01 3.6100e-01 1.1180e-05 2.7440e-05 6.1651e+01]
[1.0090e-11 3.2600e-01 3.1300e-01 1.1310e-05 2.8260e-05 6.2062e+01]
[1.3500e-12 3.6100e-01 3.5800e-01 1.1650e-05 2.8470e-05 6.1578e+01]
[3.561e-12 3.410e-01 3.350e-01 1.126e-05 2.838e-05 6.176e+01]
[5.7830e-12 3.2500e-01 3.1800e-01 1.1600e-05 3.0910e-05 6.1275e+01]
[4.6480e-12 3.3000e-01 3.1900e-01 1.1800e-05 3.2360e-05 6.1056e+01]
[4.9170e-12 3.4200e-01 3.2400e-01 1.1350e-05 3.0250e-05 6.1547e+01]
[4.0610e-12 3.5500e-01 3.3800e-01 1.1160e-05 2.7720e-05 6.1776e+01]
[3.6190e-12 3.4100e-01 3.3600e-01 1.1480e-05 2.8090e-05 6.2332e+01]
[1.6010e-12 3.5900e-01 3.4900e-01 1.1230e-05 2.9200e-05 6.1309e+01]
[9.027e-12 3.250e-01 3.060e-01 1.136e-05 3.017e-05 6.120e+01

[2.3130e-12 3.4700e-01 3.3900e-01 1.0370e-05 3.0020e-05 6.1169e+01]
[2.4480e-12 3.4600e-01 3.3800e-01 1.1150e-05 3.2910e-05 6.1223e+01]
[2.4470e-12 3.4500e-01 3.3900e-01 1.0710e-05 3.1270e-05 6.1244e+01]
[2.2760e-12 3.4600e-01 3.4000e-01 1.0680e-05 3.0790e-05 6.1207e+01]
[2.2960e-12 3.4600e-01 3.3900e-01 1.0910e-05 3.2200e-05 6.1161e+01]
[2.3040e-12 3.4700e-01 3.4000e-01 1.0760e-05 3.2480e-05 6.1224e+01]
[2.3570e-12 3.4600e-01 3.4000e-01 1.0670e-05 3.0280e-05 6.1267e+01]
[2.290e-12 3.460e-01 3.400e-01 1.012e-05 2.928e-05 6.123e+01]
[2.2050e-12 3.4800e-01 3.4100e-01 1.0630e-05 2.9900e-05 6.1265e+01]
[2.3000e-12 3.4700e-01 3.4000e-01 1.0380e-05 2.9680e-05 6.1206e+01]
[2.3980e-12 3.4500e-01 3.3900e-01 1.0570e-05 3.0440e-05 6.1267e+01]
[2.3130e-12 3.4600e-01 3.3900e-01 1.0290e-05 3.0600e-05 6.1176e+01]
[2.3060e-12 3.4700e-01 3.4000e-01 1.0640e-05 3.3220e-05 6.1206e+01]
[2.354e-12 3.460e-01 3.390e-01 1.055e-05 3.114e-05 6.116e+01]
[2.3340e-12 3.4700e-01 3.3900e-01 1.1680e-05 3.3270e-05 6.12

[2.5240e-12 3.4500e-01 3.3800e-01 1.2170e-05 3.5990e-05 6.1361e+01]
[2.3320e-12 3.4600e-01 3.3900e-01 1.1690e-05 3.3740e-05 6.1248e+01]
[2.5570e-12 3.4400e-01 3.3700e-01 1.1890e-05 3.5020e-05 6.1336e+01]
[2.6130e-12 3.4400e-01 3.3700e-01 1.1960e-05 3.7080e-05 6.1346e+01]
[2.7750e-12 3.4300e-01 3.3500e-01 1.2030e-05 3.5440e-05 6.1296e+01]
[2.506e-12 3.450e-01 3.370e-01 1.165e-05 3.514e-05 6.128e+01]
[2.6480e-12 3.4400e-01 3.3700e-01 1.2460e-05 3.5420e-05 6.1319e+01]
[2.4480e-12 3.4600e-01 3.3800e-01 1.2000e-05 3.4100e-05 6.1349e+01]
[2.7290e-12 3.4400e-01 3.3600e-01 1.2600e-05 3.6430e-05 6.1393e+01]
[2.5450e-12 3.4400e-01 3.3800e-01 1.2000e-05 3.5700e-05 6.1364e+01]
[2.7290e-12 3.4300e-01 3.3600e-01 1.1890e-05 3.5670e-05 6.1307e+01]
[2.4080e-12 3.4600e-01 3.3900e-01 1.2070e-05 3.3970e-05 6.1318e+01]
[2.4710e-12 3.4500e-01 3.3900e-01 1.1430e-05 3.1650e-05 6.1315e+01]
[2.4460e-12 3.4500e-01 3.3800e-01 1.1480e-05 3.3300e-05 6.1277e+01]
[2.571e-12 3.440e-01 3.370e-01 1.138e-05 3.320e-05 6.1

[2.645e-12 3.440e-01 3.370e-01 1.326e-05 3.748e-05 6.142e+01]
[2.6440e-12 3.4500e-01 3.3700e-01 1.3320e-05 3.7760e-05 6.1408e+01]
[2.5030e-12 3.4500e-01 3.3800e-01 1.2700e-05 3.5820e-05 6.1309e+01]
[2.6260e-12 3.4400e-01 3.3700e-01 1.2670e-05 3.6690e-05 6.1358e+01]
[2.544e-12 3.460e-01 3.380e-01 1.262e-05 3.572e-05 6.141e+01]
[2.5000e-12 3.4500e-01 3.3800e-01 1.3170e-05 3.7250e-05 6.1401e+01]
[2.634e-12 3.440e-01 3.380e-01 1.336e-05 3.679e-05 6.150e+01]
[2.460e-12 3.460e-01 3.380e-01 1.261e-05 3.580e-05 6.129e+01]
[2.4670e-12 3.4500e-01 3.3800e-01 1.2820e-05 3.5310e-05 6.1372e+01]
[2.5850e-12 3.4500e-01 3.3700e-01 1.2960e-05 3.6730e-05 6.1394e+01]
[2.7690e-12 3.4300e-01 3.3500e-01 1.3090e-05 3.6850e-05 6.1373e+01]
[2.5220e-12 3.4500e-01 3.3800e-01 1.2070e-05 3.4810e-05 6.1302e+01]
[2.5960e-12 3.4400e-01 3.3700e-01 1.2890e-05 3.6000e-05 6.1377e+01]
[2.5540e-12 3.4500e-01 3.3700e-01 1.3060e-05 3.7360e-05 6.1362e+01]
[2.5580e-12 3.4500e-01 3.3800e-01 1.3290e-05 3.7660e-05 6.1378e+01]
[2.5

# LER_20201102_testset_V04

In [39]:
# file_path version : 2020_LER_20201008_V006.xlsx

file_path = '2020_LER_20201102_testset_V04.xlsx'

data_x = pd.read_excel('../'+file_path, sheet_name='Test_set', usecols=x_cols, nrows=num_total+1, header=header)
data_y = pd.read_excel('../'+file_path, sheet_name='Test_set', usecols=y_cols, nrows=num_total+1, header=header)

# one-hot encoding (나중에 1, 0 을 인위적으로 넣어주기)
X_all , Y_all = np.zeros((num_total, num_input)), np.zeros((num_total, num_output))
X_per_cycle, Y_per_cycle = np.zeros((num_of_cycle, num_input)), np.zeros((num_of_cycle, num_output))

# DATA_X DATA_Y preprocessing

# Y: IDLO, IDHI, DIBL
data_y = data_y.drop('IDLO', axis=1)
data_y = data_y.drop('IDHI', axis=1)
data_y = data_y.drop('DIBL(mV)', axis=1)
print(data_x)
# print(data_y)

Y_all = data_y.values

# X_per_cycle

for i in range(num_of_cycle):
    X_per_cycle[i] = data_x[i*num_in_cycle:i*num_in_cycle+1].values
    
X_all = np.repeat(X_per_cycle,num_in_cycle,axis=0)  
    
# Y_per_cycle    
for i in range(num_of_cycle):
    Y_per_cycle[i] = np.mean(Y_all[i*num_in_cycle:(i+1)*num_in_cycle],axis=0)

Y_per_mean_cov = mean_cov(Y_all, num_in_cycle, num_of_cycle, num_output)
print(Y_per_mean_cov.shape)

print("============ Data load =============")

print("X data shape: ", X_all.shape, "X per cycle data shape:", X_per_cycle.shape)
print("Y data shape: ", Y_all.shape, "Y per cycle data shape:", Y_per_cycle.shape)  
print("any nan in X?: ", np.argwhere(np.isnan(X_all)))
print("any nan in Y?: ", np.argwhere(np.isnan(Y_all)))  

print("============ Data save =============")

data = []
data.append(X_all)
data.append(Y_all)
data.append(X_per_cycle)
data.append(Y_per_cycle)
data.append(Y_per_mean_cov)

# data_a = []
# data_b = []
# data_a_X_all = []
# data_a_Y_all = []
# data_b_X_all = []
# data_b_Y_all = []
# data_a_Y_cycle = []
# data_b_Y_cycle = []

# # split data

# for i in range(num_of_cycle):
    
#     choice = [i for i in range(250)]
#     mask_a = np.random.choice(choice, 125, replace=False)
    
#     # complement using set
#     choice = set(choice)
#     mask_a = set(mask_a)  
#     mask_b = choice.difference(mask_a)
#     mask_b = list(mask_b)
#     mask_a = list(mask_a)
# #     print(mask_a)
# #     print(mask_b)
    
#     target_X = X_all[250*i:250*(i+1)]
#     target_Y = Y_all[250*i:250*(i+1)]
# #     print(target_X.shape, target_Y.shape)

#     data_a_X_all.extend(target_X[mask_a])
#     data_a_Y_all.extend(target_Y[mask_a])
    
#     data_a_Y_cycle.append(np.mean(target_Y[mask_a], axis=0))
    
#     data_b_X_all.extend(target_X[mask_b])
#     data_b_Y_all.extend(target_Y[mask_b])
    
#     data_b_Y_cycle.append(np.mean(target_Y[mask_b], axis=0))

# data_a_X_all = np.array(data_a_X_all)
# data_a_Y_all = np.array(data_a_Y_all)
# data_a_Y_cyclce = np.array(data_a_Y_cycle)

# data_b_X_all = np.array(data_b_X_all)
# data_b_Y_all = np.array(data_b_Y_all)
# data_b_Y_cyclce = np.array(data_b_Y_cycle)
    
# data_a.append(data_a_X_all)
# data_a.append(data_a_Y_all)
# data_a.append(X_per_cycle)
# data_a.append(data_a_Y_cycle)

# data_b.append(data_b_X_all)
# data_b.append(data_b_Y_all)
# data_b.append(X_per_cycle)
# data_b.append(data_b_Y_cycle)

# data save

data = np.array(data)

# name = '_seed_{}'.format(seed)

print("data : data_all")
print("[saved] X data : ({},{}) X per cycle data shape : ({},{})".format(len(data[0]), len(data[0][0]), len(data[2]), len(data[2][0])))                                                                  
print("[saved] Y data : ({},{}) Y per cycle data shape : ({},{})".format(len(data[1]), len(data[1][0]), len(data[3]), len(data[3][0])))  
np.save('./'+file_path, data)

# data_a = np.array(data_a)

# print("data : data_a")
# print("[saved] X data : ({},{}) X per cycle data shape : ({},{})".format(len(data_a[0]), len(data_a[0][0]), len(data_a[2]), len(data_a[2][0])))                                                                  
# print("[saved] Y data : ({},{}) Y per cycle data shape : ({},{})".format(len(data_a[1]), len(data_a[1][0]), len(data_a[3]), len(data_a[3][0])))  
# np.save('./'+file_path+'_data_a'+name, data_a)

# data_b = np.array(data_b)

# print("data : data_b")
# print("[saved] X data : ({},{}) X per cycle data shape : ({},{})".format(len(data_b[0]), len(data_b[0][0]), len(data_b[2]), len(data_b[2][0])))                                                                  
# print("[saved] Y data : ({},{}) Y per cycle data shape : ({},{})".format(len(data_b[1]), len(data_b[1][0]), len(data_b[3]), len(data_b[3][0])))  
# np.save('./'+file_path+'_data_b'+name, data_b)

          amp.     corr.x      corr.y
0     0.624847  17.007432  173.857189
1          NaN        NaN         NaN
2          NaN        NaN         NaN
3          NaN        NaN         NaN
4          NaN        NaN         NaN
...        ...        ...         ...
2495       NaN        NaN         NaN
2496       NaN        NaN         NaN
2497       NaN        NaN         NaN
2498       NaN        NaN         NaN
2499       NaN        NaN         NaN

[2500 rows x 3 columns]
Y_all size:  (2500, 6)
(10, 27)
============ Data load =============
X data shape:  (2500, 3) X per cycle data shape: (10, 3)
Y data shape:  (2500, 6) Y per cycle data shape: (10, 6)
any nan in X?:  []
any nan in Y?:  []
============ Data save =============
data : data_all
[saved] X data : (2500,3) X per cycle data shape : (10,3)
[saved] Y data : (2500,6) Y per cycle data shape : (10,6)


In [40]:
# 검증의 시간

data = np.load(file_path+'.npy', allow_pickle=True)
    
X_all, Y_all, X_per_cycle, Y_per_cycle, Y_per_mean_cov = data[0], data[1], data[2], data[3], data[4]

print(data[0].shape, data[1].shape, data[2].shape, data[3].shape, data[4].shape)

for i in range(X_per_cycle.shape[0]):
    print(X_per_cycle[i])

total_sum = 0
for i in range(Y_all.shape[0]):
    print(i, Y_all[i])
    
# for i in range(Y_all.shape[0]):
#     if 
#     print(Y_all[i])

(2500, 3) (2500, 6) (10, 3) (10, 6) (10, 27)
[  0.62484672  17.00743225 173.85718891]
[ 0.55654727 80.91905766 80.56986979]
[ 0.50466944 48.62398816 67.99380535]
[ 0.56045148 25.42868151 69.39740975]
[ 0.63730798 69.14860261 65.33037708]
[  0.72951307  89.39156879 205.62484728]
[ 0.15273618 23.32639122 51.25326965]
[ 0.68988283 85.08858437 80.98368203]
[ 0.25925637 78.12653757 28.08679576]
[1.41144023e-01 9.67099798e+01 1.86683747e+02]
0 [1.9580e-12 4.2420e-05 1.6180e-05 3.4300e-01 3.5300e-01 6.1501e+01]
1 [6.1350e-13 3.6300e-05 1.4500e-05 3.7000e-01 3.7700e-01 6.0804e+01]
2 [7.437e-13 3.182e-05 1.391e-05 3.700e-01 3.740e-01 6.120e+01]
3 [1.4790e-12 4.1000e-05 1.5990e-05 3.5200e-01 3.5800e-01 6.1429e+01]
4 [1.9260e-12 4.3930e-05 1.7650e-05 3.4400e-01 3.5300e-01 6.1569e+01]
5 [1.1990e-12 3.6920e-05 1.4540e-05 3.5600e-01 3.6300e-01 6.1272e+01]
6 [1.6050e-12 3.6600e-05 1.3990e-05 3.4900e-01 3.5500e-01 6.1348e+01]
7 [1.5290e-12 4.0250e-05 1.5810e-05 3.5000e-01 3.5700e-01 6.1406e+01]
8 [2.2

467 [2.2590e-12 4.2670e-05 1.6700e-05 3.4200e-01 3.5000e-01 6.1852e+01]
468 [2.6660e-12 4.2170e-05 1.6920e-05 3.3900e-01 3.4700e-01 6.1918e+01]
469 [2.444e-12 4.228e-05 1.717e-05 3.390e-01 3.480e-01 6.170e+01]
470 [1.6250e-12 4.2960e-05 1.6040e-05 3.4700e-01 3.5600e-01 6.1419e+01]
471 [2.9640e-12 4.0720e-05 1.6940e-05 3.3700e-01 3.4500e-01 6.2085e+01]
472 [2.3910e-12 4.3150e-05 1.6490e-05 3.4000e-01 3.4800e-01 6.1779e+01]
473 [2.4400e-12 4.2300e-05 1.7030e-05 3.4000e-01 3.4700e-01 6.1823e+01]
474 [3.2400e-12 4.3270e-05 1.6540e-05 3.3400e-01 3.4400e-01 6.2067e+01]
475 [9.9400e-13 3.5400e-05 1.4040e-05 3.6100e-01 3.6700e-01 6.1282e+01]
476 [1.616e-12 4.049e-05 1.637e-05 3.480e-01 3.560e-01 6.147e+01]
477 [2.7220e-12 4.4310e-05 1.7560e-05 3.3700e-01 3.4700e-01 6.1915e+01]
478 [2.136e-12 4.284e-05 1.641e-05 3.420e-01 3.510e-01 6.162e+01]
479 [3.7340e-12 4.1630e-05 1.6890e-05 3.3100e-01 3.4100e-01 6.2186e+01]
480 [2.0140e-12 3.9520e-05 1.6050e-05 3.4600e-01 3.5300e-01 6.1949e+01]
481 [1.299

1033 [1.6640e-12 4.1140e-05 1.6280e-05 3.4700e-01 3.5500e-01 6.1467e+01]
1034 [1.9300e-12 4.0440e-05 1.5860e-05 3.4400e-01 3.5200e-01 6.1399e+01]
1035 [2.8660e-12 4.1290e-05 1.6060e-05 3.3800e-01 3.4600e-01 6.2106e+01]
1036 [3.2450e-12 4.0960e-05 1.5480e-05 3.3500e-01 3.4400e-01 6.2215e+01]
1037 [1.2150e-12 4.0250e-05 1.5590e-05 3.5500e-01 3.6100e-01 6.1299e+01]
1038 [1.5230e-12 3.9930e-05 1.5190e-05 3.5000e-01 3.5800e-01 6.1486e+01]
1039 [1.385e-12 3.804e-05 1.550e-05 3.540e-01 3.610e-01 6.154e+01]
1040 [2.6910e-12 4.1830e-05 1.6490e-05 3.3800e-01 3.4700e-01 6.1987e+01]
1041 [2.0720e-12 4.1510e-05 1.6260e-05 3.4200e-01 3.5100e-01 6.1524e+01]
1042 [8.790e-13 3.766e-05 1.443e-05 3.620e-01 3.680e-01 6.094e+01]
1043 [2.7430e-12 4.2090e-05 1.7230e-05 3.4000e-01 3.4700e-01 6.2173e+01]
1044 [1.8250e-12 3.9480e-05 1.5770e-05 3.4600e-01 3.5400e-01 6.1573e+01]
1045 [2.435e-12 4.145e-05 1.720e-05 3.420e-01 3.490e-01 6.204e+01]
1046 [3.0110e-12 4.1090e-05 1.7210e-05 3.3900e-01 3.4700e-01 6.2398e+

1555 [2.2870e-12 4.2990e-05 1.6560e-05 3.4100e-01 3.4800e-01 6.1681e+01]
1556 [2.4170e-12 4.3240e-05 1.6680e-05 3.4000e-01 3.4700e-01 6.1744e+01]
1557 [2.5880e-12 4.3110e-05 1.6570e-05 3.3800e-01 3.4600e-01 6.1788e+01]
1558 [2.3430e-12 4.1820e-05 1.6400e-05 3.4000e-01 3.4800e-01 6.1706e+01]
1559 [2.1970e-12 4.2930e-05 1.6310e-05 3.4100e-01 3.4900e-01 6.1593e+01]
1560 [2.4250e-12 4.2740e-05 1.6520e-05 3.3900e-01 3.4700e-01 6.1716e+01]
1561 [2.5600e-12 4.2310e-05 1.6650e-05 3.3800e-01 3.4600e-01 6.1763e+01]
1562 [2.5260e-12 4.3340e-05 1.6980e-05 3.3800e-01 3.4600e-01 6.1775e+01]
1563 [2.1040e-12 4.2300e-05 1.6340e-05 3.4200e-01 3.4900e-01 6.1571e+01]
1564 [2.1790e-12 4.2750e-05 1.6500e-05 3.4100e-01 3.4900e-01 6.1616e+01]
1565 [2.6290e-12 4.3410e-05 1.7070e-05 3.3800e-01 3.4600e-01 6.1861e+01]
1566 [2.1920e-12 4.3080e-05 1.6830e-05 3.4200e-01 3.4900e-01 6.1635e+01]
1567 [2.0230e-12 4.2990e-05 1.6410e-05 3.4300e-01 3.5000e-01 6.1509e+01]
1568 [2.295e-12 4.250e-05 1.671e-05 3.410e-01 3.480

2064 [2.4460e-12 4.2830e-05 1.6840e-05 3.3900e-01 3.4700e-01 6.1733e+01]
2065 [2.507e-12 4.197e-05 1.662e-05 3.390e-01 3.470e-01 6.185e+01]
2066 [2.6560e-12 4.3370e-05 1.6670e-05 3.3700e-01 3.4600e-01 6.1835e+01]
2067 [3.0110e-12 4.2080e-05 1.6710e-05 3.3500e-01 3.4400e-01 6.1974e+01]
2068 [2.5340e-12 4.3290e-05 1.6910e-05 3.3900e-01 3.4700e-01 6.1796e+01]
2069 [2.5520e-12 4.2950e-05 1.6890e-05 3.3900e-01 3.4700e-01 6.1833e+01]
2070 [3.0820e-12 4.3240e-05 1.6980e-05 3.3500e-01 3.4400e-01 6.2007e+01]
2071 [2.4210e-12 4.3840e-05 1.6870e-05 3.3900e-01 3.4700e-01 6.1667e+01]
2072 [2.6550e-12 4.2750e-05 1.6640e-05 3.3700e-01 3.4600e-01 6.1753e+01]
2073 [2.4800e-12 4.1790e-05 1.6690e-05 3.3900e-01 3.4700e-01 6.1792e+01]
2074 [2.9420e-12 4.3090e-05 1.6960e-05 3.3600e-01 3.4400e-01 6.2003e+01]
2075 [1.7870e-12 4.1430e-05 1.5880e-05 3.4500e-01 3.5300e-01 6.1359e+01]
2076 [2.5920e-12 4.3880e-05 1.7160e-05 3.3800e-01 3.4600e-01 6.1815e+01]
2077 [1.9420e-12 4.2950e-05 1.6590e-05 3.4400e-01 3.5200e

# 2021_RDFWFV_20210107

In [63]:
# file_path version : 2021_RDFWFV_20210107.xlsx

#     num_input = 4
#     one_hot = 2
#     num_output = 6
#     num_in_cycle = 250
#     num_of_cycle = [5, 5, 6]
#     num_total = [x*num_in_cycle for x in num_of_cycle]
#     x_cols = "C:F"
#     y_cols = "G:N"
#     onehot = "A:B"
#     header = 0

# train : AGS SD RCD RW PEAK RDF WFV 순
# 현 data : RDF WFV SD RCD RW AGS 순

file_path = '2021_RDFWFV_20210107.xlsx'

data_x = pd.read_excel('../'+file_path, sheet_name='Sheet1', usecols=x_cols, nrows=num_total+1, header=header)
data_y = pd.read_excel('../'+file_path, sheet_name='Sheet1', usecols=y_cols, nrows=num_total+1, header=header)
onehot = pd.read_excel('../'+file_path, sheet_name='Sheet1', usecols=onehot, nrows=num_total+1, header=header)

data_y = data_y.drop('Idlo', axis=1)
data_y = data_y.drop('Idhi', axis=1)

# one-hot encoding (나중에 1, 0 을 인위적으로 넣어주기)
X_all , Y_all = np.zeros((num_total, num_input)), np.zeros((num_total, num_output))
X_per_cycle, Y_per_cycle = np.zeros((sum(num_of_cycle), num_input+one_hot)), np.zeros((sum(num_of_cycle), num_output))

# DATA_X DATA_Y preprocessing

X_all = data_x.values
Y_all = data_y.values

# X_per_cycle

## (additional) AGS switch

temp_AGS = X_all[:, -1].reshape(-1, 1)
X_all = np.delete(X_all, -1, axis=1)
X_all = np.hstack((temp_AGS, X_all))
X_all = np.hstack((X_all, onehot.values))

print(X_all.shape)

for i in range(sum(num_of_cycle)):
    X_per_cycle[i] = X_all[num_in_cycle*i: num_in_cycle*i+1]
    Y_per_cycle[i] = np.mean(Y_all[num_in_cycle*i: num_in_cycle*(i+1)])

Y_per_mean_cov = mean_cov(Y_all, num_in_cycle, sum(num_of_cycle), num_output)
print(Y_per_mean_cov.shape)

print("============ Data load =============")

print("X data shape: ", X_all.shape, "X per cycle data shape:", X_per_cycle.shape)
print("Y data shape: ", Y_all.shape, "Y per cycle data shape:", Y_per_cycle.shape)  
print("any nan in X?: ", np.argwhere(np.isnan(X_all)))
print("any nan in Y?: ", np.argwhere(np.isnan(Y_all)))  

print("============ Data save =============")
    
data = []
data.append(X_all)
data.append(Y_all)
data.append(X_per_cycle)
data.append(Y_per_cycle)
data.append(Y_per_mean_cov)
    
data = np.array(data)

print("data : data_all")
print("[saved] X data : ({},{}) X per cycle data shape : ({},{})".format(len(data[0]), len(data[0][0]), len(data[2]), len(data[2][0])))                                                                  
print("[saved] Y data : ({},{}) Y per cycle data shape : ({},{})".format(len(data[1]), len(data[1][0]), len(data[3]), len(data[3][0])))  
np.save('./'+file_path, data)

(4000, 6)
Y_all size:  (4000, 6)
(16, 27)
============ Data load =============
X data shape:  (4000, 6) X per cycle data shape: (16, 6)
Y data shape:  (4000, 6) Y per cycle data shape: (16, 6)
any nan in X?:  []
any nan in Y?:  []
============ Data save =============
data : data_all
[saved] X data : (4000,6) X per cycle data shape : (16,6)
[saved] Y data : (4000,6) Y per cycle data shape : (16,6)


# 검증의 시간

In [64]:
# 검증의 시간

data = np.load(file_path+'.npy', allow_pickle=True)
    
X_all, Y_all, X_per_cycle, Y_per_cycle = data[0], data[1], data[2], data[3]

print(data[0].shape, data[1].shape, data[2].shape, data[3].shape)

for i in range(len(X_per_cycle)):
    print(X_per_cycle[i])

(4000, 6) (4000, 6) (16, 6) (16, 6)
[0.00000000e+00 5.78895870e+20 3.59561006e+19 2.88838939e-02
 1.00000000e+00 0.00000000e+00]
[0.00000000e+00 1.57080658e+19 1.19778254e+18 1.18212834e-02
 1.00000000e+00 0.00000000e+00]
[0.00000000e+00 1.97195192e+19 1.27521313e+18 3.02198477e-02
 1.00000000e+00 0.00000000e+00]
[0.00000000e+00 9.15906057e+20 1.15458844e+20 2.43231726e-02
 1.00000000e+00 0.00000000e+00]
[0.00000000e+00 4.90884204e+20 6.96683015e+19 1.91663741e-02
 1.00000000e+00 0.00000000e+00]
[6.90006771e-03 8.51545870e+19 8.25700721e+18 2.12625635e-02
 0.00000000e+00 1.00000000e+00]
[6.57265186e-03 1.74466960e+20 1.03200793e+19 2.63799659e-02
 0.00000000e+00 1.00000000e+00]
[1.85497107e-02 9.10148759e+19 8.11373190e+18 1.33503512e-02
 0.00000000e+00 1.00000000e+00]
[1.51214486e-02 7.96001377e+19 1.34431524e+19 3.71338970e-02
 0.00000000e+00 1.00000000e+00]
[5.76477267e-03 1.78541760e+19 1.45128022e+18 2.89983100e-02
 0.00000000e+00 1.00000000e+00]
[1.14038836e-02 1.31645367e+19 1.7